In [84]:
spark.stop()

In [2]:
#Lombardia

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyproj

# Create a SparkSession
spark = SparkSession.builder.appName("timeseries").config('spark.sql.shuffle.partitions',500).config('spark.driver.maxResultSize', '10G') .config("spark.driver.memory", "32g").config("spark.executor.memory", "16g").config("spark.task.maxFailures", "10").config("spark.executor.instances", "16").config("spark.local.dir", "/afs/enea.it/por/user/nafis/PFS/tmp").getOrCreate()


# Load the data from the CSV file


schema = StructType(
   [StructField('x', DoubleType(), True),
    StructField('y', DoubleType(), True),
    StructField('z', DoubleType(), True),
    StructField('time', TimestampType(), True),
    StructField('c_PM25', DoubleType(), True),
    StructField('c_PM10', DoubleType(), True),
    StructField('c_O3', DoubleType(), True),
    StructField('c_NO2', DoubleType(), True),
    StructField('geometry', StringType(), True),
    StructField('index_right', IntegerType(), True),
    StructField('OBJECTID', IntegerType(), True),
    StructField('COD_REG', IntegerType(), True),
    StructField('COD_PRO', IntegerType(), True),
    StructField('NOME', StringType(), True),
    StructField('SIGLA', StringType(), True),
    StructField('Shape_Leng', DoubleType(), True),
    StructField('Shape_Area', DoubleType(), True),
   ]
  )


df = spark.read.format("csv").load("/afs/enea.it/por/user/nafis/PFS/tmp/nafi/data_chunk/lombardia_prov/", header=True, schema=schema)

# Select the columns in the desired order
df = df.select( 'x', 'y', 'z', 'time', 'c_PM25', 'c_PM10', 'c_O3', 'c_NO2', 'geometry', 'NOME')


df = df.withColumn("x", df["x"].cast(DoubleType())) \
    .withColumn("y", df["y"].cast(DoubleType())) \
    .withColumn("z", df["z"].cast(DoubleType())) \
    .withColumn("time", df["time"].cast(TimestampType())) \
    .withColumn("c_PM25", df["c_PM25"].cast(DoubleType())) \
    .withColumn("c_PM10", df["c_PM10"].cast(DoubleType())) \
    .withColumn("c_O3", df["c_O3"].cast(DoubleType())) \
    .withColumn("c_NO2", df["c_NO2"].cast(DoubleType())) \
    .withColumn("geometry", df["geometry"].cast(StringType()))\
    .withColumn("NOME", df["NOME"].cast(StringType()))

df = df.withColumnRenamed("c_PM25", "c_PM2_5")

df.show()

df.printSchema()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/27 18:18:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/27 18:18:41 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


+--------+---------+----+-------------------+------------------+------------------+------------------+------------------+--------------------+-----+
|       x|        y|   z|               time|           c_PM2_5|            c_PM10|              c_O3|             c_NO2|            geometry| NOME|
+--------+---------+----+-------------------+------------------+------------------+------------------+------------------+--------------------+-----+
|462000.0|5016000.0|20.0|2019-11-24 00:00:00|         1.6957362|         2.7321966|          50.69704| 4.208417400000001|POINT (462000 501...|Pavia|
|462000.0|5016000.0|20.0|2019-11-24 01:00:00|1.5195319999999999|         2.5212698|         52.217495|3.6308307999999996|POINT (462000 501...|Pavia|
|462000.0|5016000.0|20.0|2019-11-24 02:00:00|1.4698639999999998|          2.540143|53.528316000000004|         3.0628326|POINT (462000 501...|Pavia|
|462000.0|5016000.0|20.0|2019-11-24 03:00:00|         1.4075043|         2.5400522|         55.269703|    

In [3]:
from pyspark.sql import functions as F
import pandas as pd

data = df.withColumnRenamed("time", "date_time")


# Convert the "date_time" column to a datetime object
data = data.withColumn("date_time", F.to_timestamp("date_time"))

# Extract the date and time values and store them in separate columns
data = data.withColumn("date", F.date_trunc("day", "date_time"))
data = data.withColumn("time", F.date_trunc("hour", "date_time"))

data = data.withColumnRenamed("date_time", "original_date_time")

# data.show()

In [4]:
from pyspark.sql.functions import col, when

# Create the season column based on the month of the date_column
data = data.withColumn(
    "season",
    when(col("original_date_time").between("2019-01-01 00:00:00", "2019-03-19 23:00:00") | col("original_date_time").between("2019-12-21 00:00:00", "2020-03-19 23:00:00") | col("original_date_time").between("2020-12-21 00:00:00", "2021-03-19 23:00:00") | col("original_date_time").between("2021-12-21 00:00:00", "2022-03-19 23:00:00") | col("original_date_time").between("2022-12-21 00:00:00", "2023-03-19 23:00:00"), "Winter")
    .when(col("original_date_time").between("2019-03-20 00:00:00", "2019-06-20 23:00:00") | col("original_date_time").between("2020-03-20 00:00:00", "2020-06-20 23:00:00") | col("original_date_time").between("2021-03-20 00:00:00", "2021-06-20 23:00:00") | col("original_date_time").between("2022-03-20 00:00:00", "2022-06-20 23:00:00") | col("original_date_time").between("2023-03-20 00:00:00", "2023-06-20 23:00:00"), "Spring")
    .when(col("original_date_time").between("2019-06-21 00:00:00", "2019-09-21 23:00:00") | col("original_date_time").between("2020-06-21 00:00:00", "2020-09-21 23:00:00") | col("original_date_time").between("2021-06-21 00:00:00", "2021-09-21 23:00:00") | col("original_date_time").between("2022-06-21 00:00:00", "2022-09-21 23:00:00") | col("original_date_time").between("2023-06-21 00:00:00", "2023-09-21 23:00:00"), "Summer")
    .when(col("original_date_time").between("2019-09-22 00:00:00", "2019-12-20 23:00:00") | col("original_date_time").between("2020-09-22 00:00:00", "2020-12-20 23:00:00") | col("original_date_time").between("2021-09-22 00:00:00", "2021-12-20 23:00:00") | col("original_date_time").between("2022-09-22 00:00:00", "2022-12-20 23:00:00") | col("original_date_time").between("2023-09-22 00:00:00", "2023-12-20 23:00:00"), "Autumn")
)


# data.show()

In [5]:
dataset = data.select(col("c_PM2_5"), col("c_PM10"), col("c_O3"), col("c_NO2"))
dataset_stat = dataset.describe()
dataset_stat.show()

+-------+------------------+------------------+--------------------+------------------+
|summary|           c_PM2_5|            c_PM10|                c_O3|             c_NO2|
+-------+------------------+------------------+--------------------+------------------+
|  count|          46463400|          46463400|            46463400|          46463400|
|   mean|12.512844285265684|13.898096026376539|   61.34618794631837|14.096244421018927|
| stddev|13.768673804829088|14.597778711778924|  30.753991251264225|18.666454366512447|
|    min|       0.019527944|       0.020731749|9.802752999999999...|       0.007907552|
|    max|          564.8532|          589.7105|            544.2421|         337.89258|
+-------+------------------+------------------+--------------------+------------------+



In [6]:
data_PM25 = data.select(col("original_date_time"),col("c_PM2_5"), col("x"), col("y"), col("NOME"))
data_PM25.show()

+-------------------+------------------+--------+---------+-----+
| original_date_time|           c_PM2_5|       x|        y| NOME|
+-------------------+------------------+--------+---------+-----+
|2019-11-24 00:00:00|         1.6957362|462000.0|5016000.0|Pavia|
|2019-11-24 01:00:00|1.5195319999999999|462000.0|5016000.0|Pavia|
|2019-11-24 02:00:00|1.4698639999999998|462000.0|5016000.0|Pavia|
|2019-11-24 03:00:00|         1.4075043|462000.0|5016000.0|Pavia|
|2019-11-24 04:00:00|1.3243808000000001|462000.0|5016000.0|Pavia|
|2019-11-24 05:00:00|1.2917066000000001|462000.0|5016000.0|Pavia|
|2019-11-24 06:00:00|         1.3729354|462000.0|5016000.0|Pavia|
|2019-11-24 07:00:00|         1.8005681|462000.0|5016000.0|Pavia|
|2019-11-24 08:00:00|         2.7273355|462000.0|5016000.0|Pavia|
|2019-11-24 09:00:00|4.3827419999999995|462000.0|5016000.0|Pavia|
|2019-11-24 10:00:00| 4.251999400000001|462000.0|5016000.0|Pavia|
|2019-11-24 11:00:00|3.8547763999999995|462000.0|5016000.0|Pavia|
|2019-11-2

In [7]:
data_PM10 = data.select(col("original_date_time"),col("c_PM10"), col("x"), col("y"), col("NOME"))
data_PM10.show()

+-------------------+------------------+--------+---------+-----+
| original_date_time|            c_PM10|       x|        y| NOME|
+-------------------+------------------+--------+---------+-----+
|2019-11-24 00:00:00|         2.7321966|462000.0|5016000.0|Pavia|
|2019-11-24 01:00:00|         2.5212698|462000.0|5016000.0|Pavia|
|2019-11-24 02:00:00|          2.540143|462000.0|5016000.0|Pavia|
|2019-11-24 03:00:00|         2.5400522|462000.0|5016000.0|Pavia|
|2019-11-24 04:00:00|         2.4868004|462000.0|5016000.0|Pavia|
|2019-11-24 05:00:00|         2.4511726|462000.0|5016000.0|Pavia|
|2019-11-24 06:00:00|         2.5612066|462000.0|5016000.0|Pavia|
|2019-11-24 07:00:00|         2.9130769|462000.0|5016000.0|Pavia|
|2019-11-24 08:00:00|         3.6733246|462000.0|5016000.0|Pavia|
|2019-11-24 09:00:00|         5.3303866|462000.0|5016000.0|Pavia|
|2019-11-24 10:00:00|         5.3588486|462000.0|5016000.0|Pavia|
|2019-11-24 11:00:00|          4.963749|462000.0|5016000.0|Pavia|
|2019-11-2

In [8]:
data_NO2 = data.select(col("original_date_time"),col("c_NO2"), col("x"), col("y"), col("NOME"))
data_NO2.show()

+-------------------+------------------+--------+---------+-----+
| original_date_time|             c_NO2|       x|        y| NOME|
+-------------------+------------------+--------+---------+-----+
|2019-11-24 00:00:00| 4.208417400000001|462000.0|5016000.0|Pavia|
|2019-11-24 01:00:00|3.6308307999999996|462000.0|5016000.0|Pavia|
|2019-11-24 02:00:00|         3.0628326|462000.0|5016000.0|Pavia|
|2019-11-24 03:00:00|          2.444656|462000.0|5016000.0|Pavia|
|2019-11-24 04:00:00|         1.9309062|462000.0|5016000.0|Pavia|
|2019-11-24 05:00:00|         1.6992023|462000.0|5016000.0|Pavia|
|2019-11-24 06:00:00|1.5480975000000001|462000.0|5016000.0|Pavia|
|2019-11-24 07:00:00|         2.1196866|462000.0|5016000.0|Pavia|
|2019-11-24 08:00:00|         4.0332394|462000.0|5016000.0|Pavia|
|2019-11-24 09:00:00|         7.3524475|462000.0|5016000.0|Pavia|
|2019-11-24 10:00:00|          6.889144|462000.0|5016000.0|Pavia|
|2019-11-24 11:00:00|          6.416884|462000.0|5016000.0|Pavia|
|2019-11-2

In [9]:
data_O3 = data.select(col("original_date_time"),col("c_O3"), col("x"), col("y"), col("NOME"))
data_O3.show()

+-------------------+------------------+--------+---------+-----+
| original_date_time|              c_O3|       x|        y| NOME|
+-------------------+------------------+--------+---------+-----+
|2019-11-24 00:00:00|          50.69704|462000.0|5016000.0|Pavia|
|2019-11-24 01:00:00|         52.217495|462000.0|5016000.0|Pavia|
|2019-11-24 02:00:00|53.528316000000004|462000.0|5016000.0|Pavia|
|2019-11-24 03:00:00|         55.269703|462000.0|5016000.0|Pavia|
|2019-11-24 04:00:00| 56.09563000000001|462000.0|5016000.0|Pavia|
|2019-11-24 05:00:00|56.588367000000005|462000.0|5016000.0|Pavia|
|2019-11-24 06:00:00|          56.55035|462000.0|5016000.0|Pavia|
|2019-11-24 07:00:00|          53.39305|462000.0|5016000.0|Pavia|
|2019-11-24 08:00:00|         45.281013|462000.0|5016000.0|Pavia|
|2019-11-24 09:00:00|         39.011024|462000.0|5016000.0|Pavia|
|2019-11-24 10:00:00|         42.592384|462000.0|5016000.0|Pavia|
|2019-11-24 11:00:00|42.843109999999996|462000.0|5016000.0|Pavia|
|2019-11-2

# Prophet Model

In [10]:
import io, os, sys, setuptools, tokenize
from pyspark.sql.types import StructType,StructField,StringType,TimestampType,ArrayType,DoubleType
from pyspark.sql.functions import current_date
from pyspark.sql.functions import pandas_udf, PandasUDFType
import matplotlib.pyplot as plt
from prophet import Prophet
from datetime import datetime
import pandas as pd
from time import time

In [79]:
# select the "first_name" and "last_name" columns and get distinct values
unique_coordinates = data.select(col("x"), col("y"), col("NOME")).distinct()

# show the unique values
unique_coordinates.show(60)

+--------+---------+-------+
|       x|        y|   NOME|
+--------+---------+-------+
|554000.0|5068000.0|Bergamo|
|598000.0|5016000.0|Brescia|
|666000.0|4996000.0|Mantova|
|566000.0|5120000.0|Sondrio|
|546000.0|5040000.0|Bergamo|
|546000.0|5064000.0|Bergamo|
|546000.0|5076000.0|Bergamo|
|598000.0|5052000.0|Brescia|
|634000.0|4996000.0|Mantova|
|638000.0|4984000.0|Mantova|
|474000.0|5000000.0|  Pavia|
|486000.0|5004000.0|  Pavia|
|542000.0|5032000.0|Cremona|
|554000.0|5060000.0|Bergamo|
|534000.0|5132000.0|Sondrio|
|474000.0|5068000.0| Varese|
|626000.0|4996000.0|Mantova|
|518000.0|5004000.0|  Pavia|
|582000.0|5148000.0|Sondrio|
|554000.0|5080000.0|Bergamo|
|638000.0|4988000.0|Mantova|
|622000.0|5064000.0|Brescia|
|518000.0|4968000.0|  Pavia|
|490000.0|5080000.0| Varese|
|506000.0|5020000.0| Milano|
|602000.0|5060000.0|Brescia|
|562000.0|5080000.0|Bergamo|
|622000.0|5052000.0|Brescia|
|626000.0|4992000.0|Mantova|
|482000.0|5104000.0| Varese|
|526000.0|5020000.0| Milano|
|562000.0|5068

PM2.5

In [12]:
from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_PM25_prophet = data_PM25.groupBy("NOME", "original_date_time").agg(avg("c_PM2_5").alias("avg_c_PM25"))
data_PM25_prophet.show()

# Apply logarithmic transformation to the aggregated column avg_c_NO2
data_PM25_prophet = data_PM25_prophet.withColumn("avg_c_PM25_log", log1p("avg_c_PM25"))

data_PM25_prophet_u = data_PM25_prophet.withColumnRenamed("avg_c_PM25_log", "y").withColumnRenamed("original_date_time", "ds")

data_PM25_prophet = data_PM25_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_PM25_prophet.printSchema()

def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    # Initiate the model
    
    # Convert "ds" column to pandas datetime format and set timezone to Europe/Rome
    group["ds"] = pd.to_datetime(group["ds"])
    
    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    
    # Fit the model
    m.fit(group[group['ds'] <= cutoff_date])
    
    # Make predictions
    forecast = m.predict(group)

    # Create the directory if it does not exist
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/PM25/'

    if not os.path.exists(path):
        os.makedirs(path)
    
   # try:
   #     fig1 = m.plot(forecast)
   #     plt.ylabel("Concentration (µg/m³)", fontweight="bold")
        # Customize x-axis label with bold font
   #     plt.xlabel("Date", fontweight="bold")
   #     fig1.suptitle(f"PM2.5 Concentration Forecast for {group['NOME'].iloc[0]}")
   #     fig1.legend(['Actual', 'Forecasted'], loc='upper right')
   #     fig1.subplots_adjust(top=0.9)
        
   #     fig2 = m.plot_components(forecast)
   #     fig2.suptitle(f"PM2.5 Concentration Component Forecast for {group['NOME'].iloc[0]}")
   #     fig2.subplots_adjust(top=0.9)

        # Save the plots as images using Matplotlib's savefig method
    #    fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/PM25/{group['NOME'].iloc[0]}_prophet_forecast.png")
    #    fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/PM25/{group['NOME'].iloc[0]}_prophet_components.png")
    
    #except Exception as e:
     #   print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    #Return the forecasted results
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

# Define the result schema
result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])

# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_PM25_prophet_train = data_PM25_prophet.filter(col('ds') <= cutoff_date)
data_PM25_prophet_test = data_PM25_prophet.filter((col('ds') > cutoff_date))

max_date = data_PM25_prophet_test.agg(max('ds')).collect()[0][0]

period = data_PM25_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_PM25 = data_PM25_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_PM25.show()


+-------+-------------------+------------------+
|   NOME| original_date_time|        avg_c_PM25|
+-------+-------------------+------------------+
|Sondrio|2019-11-24 14:00:00|0.4629849650753766|
| Varese|2020-08-04 06:00:00|1.8775407802631583|
| Varese|2021-07-11 17:00:00|1.6206620052631573|
|Brescia|2019-11-23 07:00:00|1.5688579178333328|
| Varese|2021-06-27 10:00:00| 5.575907214473685|
|  Pavia|2021-06-26 01:00:00| 2.940159353475935|
| Varese|2021-06-26 05:00:00|3.0987177657894738|
| Milano|2021-11-28 12:00:00| 11.00185946145833|
| Varese|2022-05-29 00:00:00|2.5673966217105266|
|   Como|2021-05-17 10:00:00| 2.460556795308643|
|Mantova|2019-11-23 22:00:00|2.7499121772413795|
|  Lecco|2022-06-11 19:00:00| 3.733884223529412|
|  Pavia|2022-06-11 03:00:00| 4.167420760962564|
| Varese|2021-06-26 15:00:00| 4.034924709210528|
|Sondrio|2019-11-24 04:00:00|0.3275477624623116|
|   Lodi|2020-08-04 21:00:00| 5.838125678431372|
|Brescia|2021-05-17 12:00:00| 3.178358130400001|
|  Lecco|2022-06-11 

The time used for the Spark forecast is  0.03719663619995117


Processing group: Pavia with 29862 data points                      (0 + 1) / 1]
08:28:20 - cmdstanpy - INFO - Chain [1] start processing
08:28:38 - cmdstanpy - INFO - Chain [1] done processing


+-------------------+-----+------------------+---------+----------+----------+---------+----------+-------------+-----------+
|                 ds| NOME|                 y|     yhat|yhat_upper|yhat_lower|    trend|    yearly|       weekly|      daily|
+-------------------+-----+------------------+---------+----------+----------+---------+----------+-------------+-----------+
|2019-01-01 01:00:00|Pavia| 2.762352230063346|2.5583544| 3.5408535| 1.6353321|1.7592123|0.71408874| -0.010361324| 0.09541477|
|2019-01-01 02:00:00|Pavia| 2.763489255738024| 2.570049| 3.4862893| 1.5724711|1.7597288|0.71509665|-0.0058389823| 0.10106254|
|2019-01-01 03:00:00|Pavia| 2.810006874030085|2.5803323| 3.4557323| 1.6331611|1.7602452| 0.7161072|-0.0016784158|0.105658256|
|2019-01-01 04:00:00|Pavia| 2.879674621778305|2.5896497| 3.5319276| 1.6796968|1.7607616| 0.7171204|  0.002127724| 0.10963985|
|2019-01-01 05:00:00|Pavia|  2.80605161472059| 2.594968| 3.5024784| 1.6543531|1.7612782|0.71813625|  0.005590818| 0.10

Processing group: Varese with 29862 data points


In [11]:
from pyspark.sql.functions import col
from pyspark.sql.functions import pow, sqrt

# Calculate MAE
mae_PM25 = spark_forecast_PM25.select(col("NOME"), abs(col("y") - col("yhat")).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MAE")

# Calculate MSE
mse_PM25 = spark_forecast_PM25.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MSE")

# Calculate RMSE
rmse_PM25 = spark_forecast_PM25.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                     .groupBy("NOME") \
                     .agg({"error": "mean"}) \
                     .withColumnRenamed("avg(error)", "MSE") \
                     .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE"))

# Show the results
mae_PM25.show()
mse_PM25.show()
rmse_PM25.show()

08:22:17 - cmdstanpy - INFO - Chain [1] start processing         (0 + 96) / 135]
08:22:38 - cmdstanpy - INFO - Chain [1] done processing         (97 + 38) / 135]
Processing group: Brescia with 29862 data points                    (0 + 6) / 6]
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Pavia with 29862 data points
08:22:53 - cmdstanpy - INFO - Chain [1] start processing
08:22:53 - cmdstanpy - INFO - Chain [1] start processing
08:22:53 - cmdstanpy - INFO - Chain [1] start processing
08:22:53 - cmdstanpy - INFO - Chain [1] start processing
08:22:53 - cmdstanpy - INFO - Chain [1] start processing
08:22:53 - cmdstanpy - INFO - Chain [1] start processing
08:23:06 - cmdstanpy - INFO - Chain [1] done processing
08:23:08 - cmdstanpy - INFO - Chain [1] done processing
08:23:11 - cmdstanpy - INFO - Chain [1] done processing
08:23:11 

+--------------------+-------------------+
|                NOME|                MAE|
+--------------------+-------------------+
|               Pavia| 0.3988848187069109|
|              Varese| 0.5132427317108812|
|             Brescia|0.42119224379591874|
|             Sondrio| 0.4518345470767523|
|Monza e della Bri...| 0.5737198452450825|
|                Lodi| 0.4100467279182999|
|                Como| 0.5454082754997516|
|             Bergamo| 0.4461629982114185|
|              Milano| 0.4674551905414473|
|             Cremona| 0.4473533752149172|
|             Mantova| 0.4307821850758386|
|               Lecco| 0.5432069110516565|
+--------------------+-------------------+



Processing group: Cremona with 29862 data points==============>   (13 + 1) / 14]
Processing group: Brescia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
08:24:12 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:24:12 - cmdstanpy - INFO - Chain [1] start processing
08:24:12 - cmdstanpy - INFO - Chain [1] start processing
08:24:12 - cmdstanpy - INFO - Chain [1] start processing
08:24:12 - cmdstanpy - INFO - Chain [1] start processing
08:24:12 - cmdstanpy - INFO - Chain [1] start processing
08:24:24 - cmdstanpy - INFO - Chain [1] done processing
08:24:25 - cmdstanpy - INFO - Chain [1] done processing
08:24:29 - cmdstanpy - INFO - Chain [1] done processing
08:24:31 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
08:24:32 - cmdstanpy - INFO 

+--------------------+-------------------+
|                NOME|                MSE|
+--------------------+-------------------+
|               Pavia|0.26054228295502174|
|              Varese| 0.4183188403435891|
|             Brescia|0.29374592485237644|
|             Sondrio| 0.3148080038244355|
|Monza e della Bri...| 0.5440815435088053|
|                Lodi|0.27485901727354134|
|                Como| 0.4838649345417581|
|             Bergamo|0.31509986953329455|
|              Milano| 0.3554589728558186|
|             Cremona|0.33298827884310644|
|             Mantova|0.30638187787975213|
|               Lecco|  0.474934162668179|
+--------------------+-------------------+



Processing group: Lecco with 29862 data points========>        (113 + 22) / 135]
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 29862 data points
08:25:30 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:25:30 - cmdstanpy - INFO - Chain [1] start processing
08:25:30 - cmdstanpy - INFO - Chain [1] start processing
08:25:30 - cmdstanpy - INFO - Chain [1] start processing
08:25:30 - cmdstanpy - INFO - Chain [1] start processing
08:25:30 - cmdstanpy - INFO - Chain [1] start processing
08:25:42 - cmdstanpy - INFO - Chain [1] done processing
08:25:45 - cmdstanpy - INFO - Chain [1] done processing
08:25:48 - cmdstanpy - INFO - Chain [1] done processing
08:25:48 - cmdstanpy - INFO - Chain [1] done processing
08:25:50 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della B

+--------------------+-------------------+------------------+
|                NOME|                MSE|              RMSE|
+--------------------+-------------------+------------------+
|               Pavia|0.26054228295502174|0.5104334265651318|
|              Varese| 0.4183188403435891|0.6467757264644284|
|             Brescia|0.29374592485237644|0.5419833252530712|
|             Sondrio| 0.3148080038244355|0.5610775381571032|
|Monza e della Bri...| 0.5440815435088053| 0.737618833482989|
|                Lodi|0.27485901727354134|0.5242699850969359|
|                Como| 0.4838649345417581|0.6956040069908728|
|             Bergamo|0.31509986953329455| 0.561337571816901|
|              Milano| 0.3554589728558186| 0.596203801443616|
|             Cremona|0.33298827884310644|0.5770513658619191|
|             Mantova|0.30638187787975213|0.5535177304113682|
|               Lecco|  0.474934162668179|0.6891546725287285|
+--------------------+-------------------+------------------+



In [13]:
from pyspark.sql.functions import col, pow, sqrt

# Define cutoff dates
cutoff_dates = {
    "5 Days": "2021-07-05 23:00:00",
    "30 Days": "2021-07-30 23:00:00",
    "90 Days": "2021-09-28 23:00:00",
    "180 Days": "2021-12-27 23:00:00",
    "365 Days": "2022-06-29 23:00:00"
}

# Define a list to store the evaluation results
results = []

# Compute evaluation metrics for each time period and NOME
for period, cutoff in cutoff_dates.items():
    for nome in spark_forecast_PM25.select("NOME").distinct().rdd.flatMap(lambda x: x).collect():
        # Filter the forecast data based on the cutoff date and NOME
        forecast_data_PM25 = spark_forecast_PM25.filter((col("ds") <= cutoff) & (col("NOME") == nome))

        # Calculate MAE
        mae = forecast_data_PM25.select(abs(col("y") - col("yhat")).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MAE") \
                             .collect()[0]["MAE"]

        # Calculate MSE
        mse = forecast_data_PM25.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MSE") \
                             .collect()[0]["MSE"]

        # Calculate RMSE
        rmse = forecast_data_PM25.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                              .agg({"error": "mean"}) \
                              .withColumnRenamed("avg(error)", "MSE") \
                              .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE")) \
                              .collect()[0]["RMSE"]

        # Calculate coverage
        coverage = forecast_data_PM25.filter((col("y") >= col("yhat_lower")) & (col("y") <= col("yhat_upper"))) \
                                  .count() / forecast_data_PM25.count() * 100

        # Add the results to the list
        results.append((period, nome, mse, rmse, mae, coverage))

# Create a Spark DataFrame from the results list
result_df_PM25 = spark.createDataFrame(results, ["Time", "NOME", "MSE", "RMSE", "MAE", "Coverage"])

# Show the results
result_df_PM25.show(100)

08:28:47 - cmdstanpy - INFO - Chain [1] start processing         (0 + 96) / 135]
08:29:03 - cmdstanpy - INFO - Chain [1] done processing         (18 + 96) / 135]
Processing group: Pavia with 29862 data points                      (0 + 6) / 6]
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Brescia with 29862 data points
08:29:17 - cmdstanpy - INFO - Chain [1] start processing
08:29:17 - cmdstanpy - INFO - Chain [1] start processing
08:29:17 - cmdstanpy - INFO - Chain [1] start processing
08:29:17 - cmdstanpy - INFO - Chain [1] start processing
08:29:17 - cmdstanpy - INFO - Chain [1] start processing
08:29:17 - cmdstanpy - INFO - Chain [1] start processing
08:29:28 - cmdstanpy - INFO - Chain [1] done processing
08:29:31 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Como with 29862 data points
08:29:35 - cmdsta

Processing group: Bergamo with 29862 data points======>           (11 + 3) / 14]
Processing group: Lecco with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
08:34:29 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:34:29 - cmdstanpy - INFO - Chain [1] start processing
08:34:29 - cmdstanpy - INFO - Chain [1] start processing
08:34:29 - cmdstanpy - INFO - Chain [1] start processing
08:34:29 - cmdstanpy - INFO - Chain [1] start processing
08:34:29 - cmdstanpy - INFO - Chain [1] start processing
08:34:42 - cmdstanpy - INFO - Chain [1] done processing
08:34:44 - cmdstanpy - INFO - Chain [1] done processing
08:34:46 - cmdstanpy - INFO - Chain [1] done processing
08:34:47 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
08:34:49 - cmdstanpy - INFO 

08:39:42 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:39:42 - cmdstanpy - INFO - Chain [1] start processing
08:39:42 - cmdstanpy - INFO - Chain [1] start processing
08:39:42 - cmdstanpy - INFO - Chain [1] start processing
08:39:42 - cmdstanpy - INFO - Chain [1] start processing
08:39:42 - cmdstanpy - INFO - Chain [1] start processing
08:39:55 - cmdstanpy - INFO - Chain [1] done processing
08:39:57 - cmdstanpy - INFO - Chain [1] done processing
08:39:59 - cmdstanpy - INFO - Chain [1] done processing
08:40:01 - cmdstanpy - INFO - Chain [1] done processing
08:40:01 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
08:40:03 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
08:40:04 - cmdstanpy - INFO - Chain [1] done processing
08:40:05 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
Processing group: Sondrio 

08:50:07 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:50:07 - cmdstanpy - INFO - Chain [1] start processing
08:50:07 - cmdstanpy - INFO - Chain [1] start processing
08:50:07 - cmdstanpy - INFO - Chain [1] start processing
08:50:07 - cmdstanpy - INFO - Chain [1] start processing
08:50:07 - cmdstanpy - INFO - Chain [1] start processing
08:50:20 - cmdstanpy - INFO - Chain [1] done processing
08:50:22 - cmdstanpy - INFO - Chain [1] done processing
08:50:22 - cmdstanpy - INFO - Chain [1] done processing
08:50:25 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
08:50:27 - cmdstanpy - INFO - Chain [1] done processing
08:50:28 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Sondrio with 29862 data points
08:50:28 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Como with 29862 data points
08:50:30 - cmdstanpy - INFO - Chain [1] start processing
08:50:31 - cmdstanpy - INF

08:55:27 - cmdstanpy - INFO - Chain [1] done processing
08:55:28 - cmdstanpy - INFO - Chain [1] done processing
08:55:31 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
08:55:33 - cmdstanpy - INFO - Chain [1] done processing
08:55:33 - cmdstanpy - INFO - Chain [1] done processing
08:55:35 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Como with 29862 data points
08:55:35 - cmdstanpy - INFO - Chain [1] start processing
08:55:37 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
08:55:40 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Sondrio with 29862 data points
Processing group: Milano with 29862 data points
Processing group: Varese with 29862 data points
08:55:42 - cmdstanpy - INFO - Chain [1] start processing
08:55:42 - cmdstanpy - INFO - Chain [1] start processing
08:55:43 - cmdstanpy - INFO - Chain [1] start processing
08:55:51 - cmdstanp

...

14:42:03 - cmdstanpy - INFO - Chain [1] done processing
14:42:05 - cmdstanpy - INFO - Chain [1] done processing
14:42:05 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
14:42:07 - cmdstanpy - INFO - Chain [1] done processing
14:42:08 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
Processing group: Mantova with 29862 data points
14:42:10 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Milano with 29862 data points
Processing group: Sondrio with 29862 data points
14:42:12 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Varese with 29862 data points
14:42:13 - cmdstanpy - INFO - Chain [1] start processing
14:42:13 - cmdstanpy - INFO - Chain [1] start processing
14:42:15 - cmdstanpy - INFO - Chain [1] start processing
14:42:24 - cmdstanpy - INFO - Chain [1] done processing
14:42:27 - cmdstanpy - INFO - Chain [1] done processing
14:42:27 - cmdstanp

14:47:15 - cmdstanpy - INFO - Chain [1] done processing
14:47:16 - cmdstanpy - INFO - Chain [1] done processing
14:47:16 - cmdstanpy - INFO - Chain [1] start processing
14:47:18 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Como with 29862 data points
Processing group: Mantova with 29862 data points
14:47:20 - cmdstanpy - INFO - Chain [1] start processing
14:47:20 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Sondrio with 29862 data points
Processing group: Milano with 29862 data points
14:47:24 - cmdstanpy - INFO - Chain [1] start processing
14:47:24 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Varese with 29862 data points
14:47:26 - cmdstanpy - INFO - Chain [1] start processing
14:47:34 - cmdstanpy - INFO - Chain [1] done processing
14:47:36 - cmdstanpy - INFO - Chain [1] done processing
14:47:38 - cmdstanpy - INFO - Chain [1] done processing
14:47:39 - cmdstanpy - INFO - Chain [1] done processing
14:47:39 - cmdstanpy - INF

Processing group: Como with 29862 data points
14:52:24 - cmdstanpy - INFO - Chain [1] start processing
14:52:25 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
Processing group: Milano with 29862 data points
Processing group: Sondrio with 29862 data points
14:52:29 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Varese with 29862 data points
14:52:30 - cmdstanpy - INFO - Chain [1] start processing
14:52:30 - cmdstanpy - INFO - Chain [1] start processing
14:52:30 - cmdstanpy - INFO - Chain [1] start processing
14:52:39 - cmdstanpy - INFO - Chain [1] done processing
14:52:44 - cmdstanpy - INFO - Chain [1] done processing
14:52:44 - cmdstanpy - INFO - Chain [1] done processing
14:52:45 - cmdstanpy - INFO - Chain [1] done processing
14:52:46 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
14:52:47 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Cremona with 29862 data points===>     

14:57:35 - cmdstanpy - INFO - Chain [1] start processing
14:57:35 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Milano with 29862 data points
Processing group: Sondrio with 29862 data points
Processing group: Varese with 29862 data points
14:57:39 - cmdstanpy - INFO - Chain [1] start processing
14:57:40 - cmdstanpy - INFO - Chain [1] start processing
14:57:41 - cmdstanpy - INFO - Chain [1] start processing
14:57:50 - cmdstanpy - INFO - Chain [1] done processing
14:57:50 - cmdstanpy - INFO - Chain [1] done processing
14:57:52 - cmdstanpy - INFO - Chain [1] done processing
14:57:53 - cmdstanpy - INFO - Chain [1] done processing
14:57:54 - cmdstanpy - INFO - Chain [1] done processing
14:57:57 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
Processing group: Cremona with 29862 data points==============>   (13 + 1) / 14]
Processing group: Brescia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Lecco with 2986

+--------+--------------------+-------------------+-------------------+-------------------+-----------------+
|    Time|                NOME|                MSE|               RMSE|                MAE|         Coverage|
+--------+--------------------+-------------------+-------------------+-------------------+-----------------+
|  5 Days|               Pavia|0.23246118993198644|0.48214229220426874|0.38189954120958713|94.70149253731344|
|  5 Days|              Varese| 0.3975435263997058| 0.6305105283813315| 0.5084064273967595|95.21921641791045|
|  5 Days|             Brescia|0.25107084359040843| 0.5010696993337438| 0.3993156939481304|94.70149253731344|
|  5 Days|             Sondrio|0.27378554181852754| 0.5232452023846254| 0.4248721567854352|95.06529850746269|
|  5 Days|Monza e della Bri...| 0.3908369656477005| 0.6251695495205285| 0.4967049135130239|95.01865671641792|
|  5 Days|                Lodi|0.25097823659590723| 0.5009772815167443|0.39622503624872946| 94.5195895522388|
|  5 Days|

In [ ]:
result_df_PM25.show(100)

In [ ]:
result_df_PM25_pd = result_df_PM25.pandas_api()
result_df_PM25_pd

PM10

In [11]:
from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_PM10_prophet = data_PM10.groupBy("NOME", "original_date_time").agg(avg("c_PM10").alias("avg_c_PM10"))
data_PM10_prophet.show()

# Apply logarithmic transformation to the aggregated column avg_c_NO2
data_PM10_prophet = data_PM10_prophet.withColumn("avg_c_PM10_log", log1p("avg_c_PM10"))

data_PM10_prophet_u = data_PM10_prophet.withColumnRenamed("avg_c_PM10_log", "y").withColumnRenamed("original_date_time", "ds")
# data_PM25_prophet = data_PM25_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss"))
data_PM10_prophet = data_PM10_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_PM10_prophet.printSchema()


def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    # Initiate the model
    
    # Convert "ds" column to pandas datetime format and set timezone to Europe/Rome
    #group["ds"] = pd.to_datetime(group["ds"]).dt.tz_localize('Europe/Rome').dt.tz_convert(pytz.UTC)
    group["ds"] = pd.to_datetime(group["ds"])

    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    #m.add_country_holidays(country_name="IT")
    
    # Fit the model
    m.fit(group[group['ds'] <= cutoff_date])
    
    
    # Make predictions
    forecast = m.predict(group)

#     future_forecast = m.predict(m.make_future_dataframe(periods=period, freq="H", include_history=True).loc[group['ds'] > cutoff_date])

    # Create the directory if it does not exist
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/PM10/'

    if not os.path.exists(path):
        os.makedirs(path)
    
  #  try:
  #      fig1 = m.plot(forecast)
  #      plt.ylabel("Concentration (µg/m³)", fontweight="bold")
  #      plt.xlabel("Date", fontweight="bold")
  #      fig1.suptitle(f"PM10 Concentration Forecast for {group['NOME'].iloc[0]}")
  #      fig1.legend(['Actual', 'Forecasted'], loc='upper right')
   #     fig1.subplots_adjust(top=0.9)
        
   #     fig2 = m.plot_components(forecast)
    #    fig2.suptitle(f"PM10 Concentration Component Forecast for {group['NOME'].iloc[0]}")
    #    fig2.subplots_adjust(top=0.9)

         #Save the plots as images using Matplotlib's savefig method
     #   fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/PM10/{group['NOME'].iloc[0]}_prophet_forecast.png")
      #  fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/PM10/{group['NOME'].iloc[0]}_prophet_components.png")
   # except Exception as e:
    #    print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    #Return the forecasted results
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

# Define the restult schema
result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])

# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_PM10_prophet_train = data_PM10_prophet.filter(col('ds') <= cutoff_date)
data_PM10_prophet_test = data_PM10_prophet.filter((col('ds') > cutoff_date))

max_date = data_PM10_prophet_test.agg(max('ds')).collect()[0][0]

period = data_PM10_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_PM10 = data_PM10_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_PM10.show()

+-------+-------------------+------------------+
|   NOME| original_date_time|        avg_c_PM10|
+-------+-------------------+------------------+
|Sondrio|2019-11-24 14:00:00| 1.532554649246231|
| Varese|2020-08-04 06:00:00|2.2014404381578947|
| Varese|2021-07-11 17:00:00|1.7296184092105258|
|Brescia|2019-11-23 07:00:00|1.8590820483666675|
| Varese|2021-06-27 10:00:00|6.2345089894736825|
|  Pavia|2021-06-26 01:00:00| 4.496175512834227|
| Varese|2021-06-26 05:00:00|3.5258748842105265|
| Milano|2021-11-28 12:00:00|11.555928287500002|
| Varese|2022-05-29 00:00:00|2.8523651335526323|
|   Como|2021-05-17 10:00:00|3.9270444765432106|
|Mantova|2019-11-23 22:00:00| 4.798481184827586|
|  Lecco|2022-06-11 19:00:00| 4.521177043137254|
|  Pavia|2022-06-11 03:00:00|  4.70305849144385|
| Varese|2021-06-26 15:00:00| 4.952178510526315|
|Sondrio|2019-11-24 04:00:00|1.0391447534673364|
|   Lodi|2020-08-04 21:00:00| 8.042946805882352|
|Brescia|2021-05-17 12:00:00| 3.773742657633331|
|  Lecco|2022-06-11 

The time used for the Spark forecast is  2.603619337081909


Processing group: Pavia with 29862 data points                      (0 + 1) / 1]
18:21:24 - cmdstanpy - INFO - Chain [1] start processing
18:21:38 - cmdstanpy - INFO - Chain [1] done processing


+-------------------+-----+------------------+---------+----------+----------+---------+----------+------------+------------+
|                 ds| NOME|                 y|     yhat|yhat_upper|yhat_lower|    trend|    yearly|      weekly|       daily|
+-------------------+-----+------------------+---------+----------+----------+---------+----------+------------+------------+
|2019-01-01 01:00:00|Pavia|2.8261441759659585|2.6491697| 3.5606139| 1.7630733|1.8722717| 0.6853495|4.6180576E-4|  0.09108679|
|2019-01-01 02:00:00|Pavia|2.8291165734595776|2.6575744|  3.554952| 1.6940199|1.8727866| 0.6862377| 0.005272754|  0.09327727|
|2019-01-01 03:00:00|Pavia|2.8760534203831116|2.6648011| 3.6058884| 1.6816407|1.8733016| 0.6871287| 0.009662997|  0.09470771|
|2019-01-01 04:00:00|Pavia|2.9384560067237255| 2.670035| 3.6021457| 1.7303532|1.8738167| 0.6880225| 0.013636688|  0.09455906|
|2019-01-01 05:00:00|Pavia| 2.861737345339265| 2.671015| 3.5627372|  1.781038|1.8743317| 0.6889189| 0.017202562|  0.09

Processing group: Varese with 29862 data points


In [15]:
from pyspark.sql.functions import col
from pyspark.sql.functions import pow, sqrt 
from prophet.diagnostics import performance_metrics


# Calculate MAE
mae_PM10 = spark_forecast_PM10.select(col("NOME"), abs(col("y") - col("yhat")).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MAE")


# Calculate MSE
mse_PM10 = spark_forecast_PM10.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MSE")

# Calculate RMSE
rmse_PM10 = spark_forecast_PM10.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                     .groupBy("NOME") \
                     .agg({"error": "mean"}) \
                     .withColumnRenamed("avg(error)", "MSE") \
                     .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE"))

# Show the results
mae_PM10.show()
mse_PM10.show()
rmse_PM10.show()

08:19:33 - cmdstanpy - INFO - Chain [1] start processing         (0 + 96) / 135]
Processing group: Lecco with 29862 data points                      (0 + 6) / 6]
Processing group: Pavia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Bergamo with 29862 data points
08:20:03 - cmdstanpy - INFO - Chain [1] done processing
08:20:06 - cmdstanpy - INFO - Chain [1] start processing
08:20:06 - cmdstanpy - INFO - Chain [1] start processing
08:20:06 - cmdstanpy - INFO - Chain [1] start processing
08:20:06 - cmdstanpy - INFO - Chain [1] start processing
08:20:06 - cmdstanpy - INFO - Chain [1] start processing
08:20:06 - cmdstanpy - INFO - Chain [1] start processing
08:20:21 - cmdstanpy - INFO - Chain [1] done processing
08:20:21 - cmdstanpy - INFO - Chain [1] done processing
08:20:24 - cmdstanpy - INFO - Chain [1] done processing
08:20:25 - cmdstanpy - INFO - Ch

+--------------------+-------------------+
|                NOME|                MAE|
+--------------------+-------------------+
|               Pavia|0.39601142371537246|
|              Varese| 0.5011988050097763|
|             Brescia| 0.4256356271448191|
|             Sondrio|0.43818229119099283|
|Monza e della Bri...| 0.5526475645829463|
|                Lodi| 0.4203591667432596|
|                Como| 0.5276834920867571|
|             Bergamo| 0.4463247871708737|
|              Milano| 0.4739096002881601|
|             Cremona| 0.4323231944515459|
|             Mantova|0.41874014997399495|
|               Lecco| 0.5388373898146389|
+--------------------+-------------------+



Processing group: Cremona with 29862 data points==============>   (13 + 1) / 14]
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Brescia with 29862 data points
08:21:26 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:21:26 - cmdstanpy - INFO - Chain [1] start processing
08:21:26 - cmdstanpy - INFO - Chain [1] start processing
08:21:26 - cmdstanpy - INFO - Chain [1] start processing
08:21:26 - cmdstanpy - INFO - Chain [1] start processing
08:21:26 - cmdstanpy - INFO - Chain [1] start processing
08:21:41 - cmdstanpy - INFO - Chain [1] done processing
08:21:42 - cmdstanpy - INFO - Chain [1] done processing
08:21:45 - cmdstanpy - INFO - Chain [1] done processing
08:21:46 - cmdstanpy - INFO - Chain [1] done processing
08:21:46 - cmdstanpy - INFO - Chain [1] done processing
08:21:47 - cmdstanpy - INFO - Chain

+--------------------+-------------------+
|                NOME|                MSE|
+--------------------+-------------------+
|               Pavia|0.25612014919380804|
|              Varese| 0.4027371356605693|
|             Brescia|0.29965821839140977|
|             Sondrio| 0.3115964806367943|
|Monza e della Bri...|0.49980392305517785|
|                Lodi|0.29069511104185936|
|                Como| 0.4482202191840605|
|             Bergamo| 0.3232276200899263|
|              Milano| 0.3666746026965614|
|             Cremona|0.30876722315226246|
|             Mantova|0.28856363907305804|
|               Lecco| 0.4713292861820906|
+--------------------+-------------------+



Processing group: Lodi with 29862 data points=======>          (108 + 27) / 135]
Processing group: Bergamo with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
08:22:49 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:22:49 - cmdstanpy - INFO - Chain [1] start processing
08:22:49 - cmdstanpy - INFO - Chain [1] start processing
08:22:49 - cmdstanpy - INFO - Chain [1] start processing
08:22:49 - cmdstanpy - INFO - Chain [1] start processing
08:22:49 - cmdstanpy - INFO - Chain [1] start processing
08:23:05 - cmdstanpy - INFO - Chain [1] done processing
08:23:06 - cmdstanpy - INFO - Chain [1] done processing
08:23:09 - cmdstanpy - INFO - Chain [1] done processing
08:23:09 - cmdstanpy - INFO - Chain [1] done processing
08:23:09 - cmdstanpy - INFO - Chain [1] done processing
08:23:10 - cmdstanpy - INFO - Ch

+--------------------+-------------------+------------------+
|                NOME|                MSE|              RMSE|
+--------------------+-------------------+------------------+
|               Pavia|0.25612014919380804|0.5060831445462376|
|              Varese| 0.4027371356605693|0.6346157385856179|
|             Brescia|0.29965821839140977|0.5474104660959724|
|             Sondrio| 0.3115964806367943|0.5582082771124003|
|Monza e della Bri...|0.49980392305517785|0.7069681202537904|
|                Lodi|0.29069511104185936|0.5391614888341519|
|                Como| 0.4482202191840605|0.6694925086840483|
|             Bergamo| 0.3232276200899263|0.5685311074074366|
|              Milano| 0.3666746026965614|0.6055366237450559|
|             Cremona|0.30876722315226246|0.5556682671812945|
|             Mantova|0.28856363907305804|0.5371811976168358|
|               Lecco| 0.4713292861820906| 0.686534257107459|
+--------------------+-------------------+------------------+



In [12]:
from pyspark.sql.functions import col, pow, sqrt

# Define cutoff dates
cutoff_dates = {
    "5 Days": "2021-07-05 23:00:00",
    "30 Days": "2021-07-30 23:00:00",
    "90 Days": "2021-09-28 23:00:00",
    "180 Days": "2021-12-27 23:00:00",
    "365 Days": "2022-06-29 23:00:00"
}

# Define a list to store the evaluation results
results = []

# Compute evaluation metrics for each time period and NOME
for period, cutoff in cutoff_dates.items():
    for nome in spark_forecast_PM10.select("NOME").distinct().rdd.flatMap(lambda x: x).collect():
        # Filter the forecast data based on the cutoff date and NOME
        forecast_data_PM10 = spark_forecast_PM10.filter((col("ds") <= cutoff) & (col("NOME") == nome))

        # Calculate MAE
        mae = forecast_data_PM10.select(abs(col("y") - col("yhat")).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MAE") \
                             .collect()[0]["MAE"]

        # Calculate MSE
        mse = forecast_data_PM10.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MSE") \
                             .collect()[0]["MSE"]

        # Calculate RMSE
        rmse = forecast_data_PM10.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                              .agg({"error": "mean"}) \
                              .withColumnRenamed("avg(error)", "MSE") \
                              .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE")) \
                              .collect()[0]["RMSE"]

        # Calculate coverage
        coverage = forecast_data_PM10.filter((col("y") >= col("yhat_lower")) & (col("y") <= col("yhat_upper"))) \
                                  .count() / forecast_data_PM10.count() * 100

        # Add the results to the list
        results.append((period, nome, mse, rmse, mae, coverage))

# Create a Spark DataFrame from the results list
result_df_PM10 = spark.createDataFrame(results, ["Time", "NOME", "MSE", "RMSE", "MAE", "Coverage"])

# Show the results
result_df_PM10.show(100)

18:21:46 - cmdstanpy - INFO - Chain [1] start processing         (0 + 96) / 135]
18:22:05 - cmdstanpy - INFO - Chain [1] done processing         (97 + 38) / 135]
Processing group: Lecco with 29862 data points                      (0 + 6) / 6]
Processing group: Brescia with 29862 data pointsProcessing group: Pavia with 29862 data pointsProcessing group: Bergamo with 29862 data points


Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
18:22:12 - cmdstanpy - INFO - Chain [1] start processing
18:22:12 - cmdstanpy - INFO - Chain [1] start processing
18:22:12 - cmdstanpy - INFO - Chain [1] start processing
18:22:12 - cmdstanpy - INFO - Chain [1] start processing
18:22:12 - cmdstanpy - INFO - Chain [1] start processing
18:22:12 - cmdstanpy - INFO - Chain [1] start processing
18:22:23 - cmdstanpy - INFO - Chain [1] done processing
18:22:27 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Mantova with 29862 data points
18:22:30 - cmd

18:26:51 - cmdstanpy - INFO - Chain [1] done processing
18:26:51 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Lecco with 29862 data points==========>      (118 + 17) / 135]
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Cremona with 29862 data points
18:27:22 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
18:27:22 - cmdstanpy - INFO - Chain [1] start processing
18:27:22 - cmdstanpy - INFO - Chain [1] start processing
18:27:22 - cmdstanpy - INFO - Chain [1] start processing
18:27:22 - cmdstanpy - INFO - Chain [1] start processing
18:27:22 - cmdstanpy - INFO - Chain [1] start processing
18:27:36 - cmdstanpy - INFO - Chain [1] done processing
18:27:37 - cmdstanpy - INFO - Chain [1] done processing
18:27:39 - cmdstanpy - INFO - Chain [1] done processing
18:27:40 - cmdstanpy - INFO - Cha

Processing group: Brescia with 29862 data points======>           (11 + 3) / 14]
Processing group: Cremona with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Lecco with 29862 data points
18:37:41 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
18:37:41 - cmdstanpy - INFO - Chain [1] start processing
18:37:41 - cmdstanpy - INFO - Chain [1] start processing
18:37:41 - cmdstanpy - INFO - Chain [1] start processing
18:37:41 - cmdstanpy - INFO - Chain [1] start processing
18:37:41 - cmdstanpy - INFO - Chain [1] start processing
18:37:54 - cmdstanpy - INFO - Chain [1] done processing
18:37:55 - cmdstanpy - INFO - Chain [1] done processing
18:37:56 - cmdstanpy - INFO - Chain [1] done processing
18:37:58 - cmdstanpy - INFO - Chain [1] done processing
18:37:59 - cmdstanpy - INFO - Chain [1] done processing
18:38:01 - cmdstanpy - INFO - Chain

Processing group: Lecco with 29862 data points==========>      (117 + 18) / 135]
Processing group: Brescia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lodi with 29862 data points
18:48:07 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
18:48:07 - cmdstanpy - INFO - Chain [1] start processing
18:48:07 - cmdstanpy - INFO - Chain [1] start processing
18:48:07 - cmdstanpy - INFO - Chain [1] start processing
18:48:07 - cmdstanpy - INFO - Chain [1] start processing
18:48:07 - cmdstanpy - INFO - Chain [1] start processing
18:48:20 - cmdstanpy - INFO - Chain [1] done processing
18:48:21 - cmdstanpy - INFO - Chain [1] done processing
18:48:26 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Sondrio with 29862 data points
18:48:26 - cmdstanpy - INFO - Chain [1] done processing
18:48:27 - cmdstanpy - INFO - Chain [1] 

Processing group: Bergamo with 29862 data points=>             (101 + 34) / 135]
Processing group: Lodi with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
18:58:30 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
18:58:30 - cmdstanpy - INFO - Chain [1] start processing
18:58:30 - cmdstanpy - INFO - Chain [1] start processing
18:58:30 - cmdstanpy - INFO - Chain [1] start processing
18:58:30 - cmdstanpy - INFO - Chain [1] start processing
18:58:30 - cmdstanpy - INFO - Chain [1] start processing
18:58:43 - cmdstanpy - INFO - Chain [1] done processing
18:58:43 - cmdstanpy - INFO - Chain [1] done processing
18:58:45 - cmdstanpy - INFO - Chain [1] done processing
18:58:46 - cmdstanpy - INFO - Chain [1] done processing
18:58:47 - cmdstanpy - INFO - Chain [1] done processing
18:58:48 - cmdstanpy - INFO - Chain

...

00:38:27 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Lecco with 29862 data points====>            (103 + 32) / 135]
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
00:38:58 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
00:38:58 - cmdstanpy - INFO - Chain [1] start processing
00:38:58 - cmdstanpy - INFO - Chain [1] start processing
00:38:58 - cmdstanpy - INFO - Chain [1] start processing
00:38:58 - cmdstanpy - INFO - Chain [1] start processing
00:38:58 - cmdstanpy - INFO - Chain [1] start processing
00:39:11 - cmdstanpy - INFO - Chain [1] done processing
00:39:12 - cmdstanpy - INFO - Chain [1] done processing
00:39:14 - cmdstanpy - INFO - Chain [1] done processing
00:39:15 - cmdstanpy - INFO - Chain [1] done processing
00:39:15 - cmdstanpy - INFO - Cha

00:48:50 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Lecco with 29862 data points============>       (12 + 2) / 14]
Processing group: Lodi with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
00:49:20 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
00:49:20 - cmdstanpy - INFO - Chain [1] start processing
00:49:20 - cmdstanpy - INFO - Chain [1] start processing
00:49:20 - cmdstanpy - INFO - Chain [1] start processing
00:49:20 - cmdstanpy - INFO - Chain [1] start processing
00:49:20 - cmdstanpy - INFO - Chain [1] start processing
00:49:33 - cmdstanpy - INFO - Chain [1] done processing
00:49:35 - cmdstanpy - INFO - Chain [1] done processing
00:49:36 - cmdstanpy - INFO - Chain [1] done processing
00:49:37 - cmdstanpy - INFO - Chain [1] done processing
00:49:38 - cmdstanpy - INFO - Cha

Processing group: Cremona with 29862 data points=====>         (109 + 26) / 135]
Processing group: Brescia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Pavia with 29862 data points
00:54:34 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
00:54:34 - cmdstanpy - INFO - Chain [1] start processing
00:54:34 - cmdstanpy - INFO - Chain [1] start processing
00:54:34 - cmdstanpy - INFO - Chain [1] start processing
00:54:34 - cmdstanpy - INFO - Chain [1] start processing
00:54:34 - cmdstanpy - INFO - Chain [1] start processing
00:54:49 - cmdstanpy - INFO - Chain [1] done processing
00:54:50 - cmdstanpy - INFO - Chain [1] done processing
00:54:53 - cmdstanpy - INFO - Chain [1] done processing
00:54:53 - cmdstanpy - INFO - Chain [1] done processing
00:54:54 - cmdstanpy - INFO - Chain [1] done processing
00:54:55 - cmdstanpy - INFO - Chain

Processing group: Cremona with 29862 data points========>      (117 + 18) / 135]
Processing group: Lecco with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Pavia with 29862 data points
01:05:08 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
01:05:08 - cmdstanpy - INFO - Chain [1] start processing
01:05:08 - cmdstanpy - INFO - Chain [1] start processing
01:05:08 - cmdstanpy - INFO - Chain [1] start processing
01:05:08 - cmdstanpy - INFO - Chain [1] start processing
01:05:08 - cmdstanpy - INFO - Chain [1] start processing
01:05:22 - cmdstanpy - INFO - Chain [1] done processing
01:05:23 - cmdstanpy - INFO - Chain [1] done processing
01:05:25 - cmdstanpy - INFO - Chain [1] done processing
01:05:26 - cmdstanpy - INFO - Chain [1] done processing
01:05:26 - cmdstanpy - INFO - Chain [1] done processing
01:05:27 - cmdstanpy - INFO - Chain

+--------+--------------------+-------------------+-------------------+-------------------+-----------------+
|    Time|                NOME|                MSE|               RMSE|                MAE|         Coverage|
+--------+--------------------+-------------------+-------------------+-------------------+-----------------+
|  5 Days|               Pavia| 0.2152419518964988|0.46394175485345013|0.37004821642519176|94.88805970149254|
|  5 Days|              Varese| 0.3777545554414288| 0.6146174057423275| 0.4971025926003706|95.10261194029852|
|  5 Days|             Brescia|0.23057629530485566| 0.4801836058268292| 0.3834494727590663|94.85074626865672|
|  5 Days|             Sondrio| 0.2638669860636993|  0.513679847827126|0.41549712372427144|94.99067164179105|
|  5 Days|Monza e della Bri...| 0.3673753534219255| 0.6061149671654096|0.48257724940682345|95.01865671641792|
|  5 Days|                Lodi| 0.2296273639262174|0.47919449488304583|0.38064205133666756|94.61287313432835|
|  5 Days|

In [ ]:
result_df_PM10_pd = result_df_PM10.pandas_api()
result_df_PM10_pd

NO2

In [15]:
from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os
from pyspark.sql.functions import log1p

# Aggregate data by day
#data_NO2_daily = data_NO2.withColumn("day", date_trunc("day", col("original_date_time"))) \
#                         .groupBy("NOME", "day") \
#                         .agg(avg("c_NO2").alias("avg_c_NO2"))

#data_NO2_daily.show()

# Rename columns
#data_NO2_prophet_u = data_NO2_daily.withColumnRenamed("avg_c_NO2", "y").withColumnRenamed("day", "ds")

# Convert to timestamp and adjust timezone
#data_NO2_prophet = data_NO2_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
#                                     .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

#data_NO2_prophet.printSchema()

data_NO2_prophet = data_NO2.groupBy("NOME", "original_date_time").agg(avg("c_NO2").alias("avg_c_NO2"))
data_NO2_prophet.show()

#data_NO2_prophet_u = data_NO2_prophet.withColumnRenamed("avg_c_NO2", "y").withColumnRenamed("original_date_time", "ds")


# Apply logarithmic transformation to the aggregated column avg_c_NO2
data_NO2_prophet = data_NO2_prophet.withColumn("avg_c_NO2_log", log1p("avg_c_NO2"))

# Rename the transformed column
data_NO2_prophet = data_NO2_prophet.withColumnRenamed("avg_c_NO2_log", "y") \
                                   .withColumnRenamed("original_date_time", "ds")

data_NO2_prophet = data_NO2_prophet.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))


data_NO2_prophet.printSchema()
data_NO2_prophet.show()



def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    group["ds"] = pd.to_datetime(group["ds"])
    
    
    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    
    m.fit(group[group['ds'] <= cutoff_date])
    
    forecast = m.predict(group)
    
    # Filter out non-zero values in the forecast
    #forecast_filtered = forecast[(forecast['yhat'] >= 0) & (forecast['yhat_lower'] >= 0) & (forecast['yhat_upper'] >= 0)]
    
    
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/NO2/'

    if not os.path.exists(path):
        os.makedirs(path)
    
  #  try:
   #     fig1 = m.plot(forecast)
    #    plt.ylabel("Concentration (µg/m3)", fontweight="bold")
     #   plt.xlabel("Date", fontweight="bold")
      #  fig1.suptitle(f"NO2 Concentration Forecast for {group['NOME'].iloc[0]}")
       # fig1.legend(['Actual', 'Forecasted'], loc='upper right')
        #fig1.subplots_adjust(top=0.9)

        
        
       # fig2 = m.plot_components(forecast)
       # fig2.suptitle(f"NO2 Concentration Component Forecast for {group['NOME'].iloc[0]}")
       # fig2.subplots_adjust(top=0.9)

       # fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/NO2/{group['NOME'].iloc[0]}_prophet_forecast.png")
       # fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/NO2/{group['NOME'].iloc[0]}_prophet_components.png")
   # except Exception as e:
    #    print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
        
    
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])

# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_NO2_prophet_train = data_NO2_prophet.filter(col('ds') <= cutoff_date)
data_NO2_prophet_test = data_NO2_prophet.filter((col('ds') > cutoff_date))

max_date = data_NO2_prophet_test.agg(max('ds')).collect()[0][0]

period = data_NO2_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_NO2 = data_NO2_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_NO2.show()


+-------+-------------------+------------------+
|   NOME| original_date_time|         avg_c_NO2|
+-------+-------------------+------------------+
|Sondrio|2019-11-24 14:00:00|0.6643196169346735|
| Varese|2020-08-04 06:00:00| 5.275944802631578|
| Varese|2021-07-11 17:00:00|2.3521740331578953|
|Brescia|2019-11-23 07:00:00|3.0555828133666694|
| Varese|2021-06-27 10:00:00| 8.993028231052632|
|  Pavia|2021-06-26 01:00:00| 3.174677009037433|
| Varese|2021-06-26 05:00:00|  4.15398733736842|
| Milano|2021-11-28 12:00:00|18.202434293750002|
| Varese|2022-05-29 00:00:00|2.8546385980263156|
|   Como|2021-05-17 10:00:00| 6.047174076296298|
|Mantova|2019-11-23 22:00:00| 6.472082522758619|
|  Lecco|2022-06-11 19:00:00|3.8186895750980407|
|  Pavia|2022-06-11 03:00:00|4.2664332326203205|
| Varese|2021-06-26 15:00:00| 3.133505573026316|
|Sondrio|2019-11-24 04:00:00|0.4946508185427134|
|   Lodi|2020-08-04 21:00:00| 34.30330264705882|
|Brescia|2021-05-17 12:00:00|2.1098869339400013|
|  Lecco|2022-06-11 

+-------+-------------------+------------------+-------------------+
|   NOME|                 ds|         avg_c_NO2|                  y|
+-------+-------------------+------------------+-------------------+
|Sondrio|2019-11-24 15:00:00|0.6643196169346735| 0.5094164014392784|
| Varese|2020-08-04 08:00:00| 5.275944802631578| 1.8367240398403282|
| Varese|2021-07-11 19:00:00|2.3521740331578953| 1.2096091004719312|
|Brescia|2019-11-23 08:00:00|3.0555828133666694| 1.4000944043337502|
| Varese|2021-06-27 12:00:00| 8.993028231052632|  2.301887672958483|
|  Pavia|2021-06-26 03:00:00| 3.174677009037433|  1.429036992205333|
| Varese|2021-06-26 07:00:00|  4.15398733736842| 1.6397706553719789|
| Milano|2021-11-28 13:00:00|18.202434293750002|  2.955037057129866|
| Varese|2022-05-29 02:00:00|2.8546385980263156| 1.3492772537298816|
|   Como|2021-05-17 12:00:00| 6.047174076296298| 1.9526266962245933|
|Mantova|2019-11-23 23:00:00| 6.472082522758619| 2.0111737451319645|
|  Lecco|2022-06-11 21:00:00|3.818

The time used for the Spark forecast is  0.0389249324798584


Processing group: Pavia with 29862 data points                      (0 + 1) / 1]
07:57:46 - cmdstanpy - INFO - Chain [1] start processing
07:57:57 - cmdstanpy - INFO - Chain [1] done processing


+-------------------+-----+------------------+---------+----------+----------+---------+----------+-----------+-------------+
|                 ds| NOME|                 y|     yhat|yhat_upper|yhat_lower|    trend|    yearly|     weekly|        daily|
+-------------------+-----+------------------+---------+----------+----------+---------+----------+-----------+-------------+
|2019-01-01 01:00:00|Pavia| 3.364060919731788|3.5590816| 4.4179006| 2.7241468|2.2092843|0.74671125| 0.15687892|   0.44620705|
|2019-01-01 02:00:00|Pavia| 3.359712772477831|3.5162702|  4.361835| 2.6083496|2.2096746|  0.746776| 0.16044094|   0.39937863|
|2019-01-01 03:00:00|Pavia|3.3562234354558336| 3.461882|  4.348633| 2.5354877| 2.210065| 0.7468423|   0.162769|   0.34220597|
|2019-01-01 04:00:00|Pavia|3.3093035128516446|3.3891768| 4.2674694| 2.5037506| 2.210455| 0.7469102| 0.16394956|   0.26786208|
|2019-01-01 05:00:00|Pavia| 3.158742259713706|3.3038254|  4.144061|  2.465234|2.2108452|0.74697965| 0.16407739|   0.18

Processing group: Varese with 29862 data points


In [14]:
from pyspark.sql.functions import col
from pyspark.sql.functions import pow, sqrt
from prophet.diagnostics import performance_metrics


# Calculate MAE
mae_NO2 = spark_forecast_NO2.select(col("NOME"), abs(col("y") - col("yhat")).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MAE")

# Calculate MSE
mse_NO2 = spark_forecast_NO2.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MSE")

# Calculate RMSE
rmse_NO2 = spark_forecast_NO2.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                     .groupBy("NOME") \
                     .agg({"error": "mean"}) \
                     .withColumnRenamed("avg(error)", "MSE") \
                     .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE"))

# Show the results
mae_NO2.show()
mse_NO2.show()
rmse_NO2.show()

07:09:24 - cmdstanpy - INFO - Chain [1] start processing         (0 + 96) / 135]
07:09:45 - cmdstanpy - INFO - Chain [1] done processing        (107 + 28) / 135]
Processing group: Lodi with 29862 data points                       (0 + 6) / 6]
Processing group: Lecco with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
07:09:52 - cmdstanpy - INFO - Chain [1] start processing
07:09:52 - cmdstanpy - INFO - Chain [1] start processing
07:09:52 - cmdstanpy - INFO - Chain [1] start processing
07:09:52 - cmdstanpy - INFO - Chain [1] start processing
07:09:52 - cmdstanpy - INFO - Chain [1] start processing
07:09:52 - cmdstanpy - INFO - Chain [1] start processing
07:10:02 - cmdstanpy - INFO - Chain [1] done processing
07:10:04 - cmdstanpy - INFO - Chain [1] done processing
07:10:05 - cmdstanpy - INFO - Chain [1] done processing
07:10:

+--------------------+-------------------+
|                NOME|                MAE|
+--------------------+-------------------+
|               Pavia| 0.3928372052638267|
|              Varese| 0.4780412256793591|
|             Brescia|0.34080538507346875|
|             Sondrio| 0.3402616502251795|
|Monza e della Bri...| 0.5240357695701044|
|                Lodi| 0.3956175466595447|
|                Como| 0.5377956615447079|
|             Bergamo|  0.384215338276944|
|              Milano| 0.4286919271198144|
|             Cremona| 0.3579551730787786|
|             Mantova|0.34893409700759315|
|               Lecco| 0.5092796043063625|
+--------------------+-------------------+



Processing group: Lodi with 29862 data points======>           (105 + 30) / 135]
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lecco with 29862 data points
07:11:11 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
07:11:11 - cmdstanpy - INFO - Chain [1] start processing
07:11:11 - cmdstanpy - INFO - Chain [1] start processing
07:11:11 - cmdstanpy - INFO - Chain [1] start processing
07:11:12 - cmdstanpy - INFO - Chain [1] start processing
07:11:12 - cmdstanpy - INFO - Chain [1] start processing
07:11:23 - cmdstanpy - INFO - Chain [1] done processing
07:11:24 - cmdstanpy - INFO - Chain [1] done processing
07:11:26 - cmdstanpy - INFO - Chain [1] done processing
07:11:28 - cmdstanpy - INFO - Chain [1] done processing
07:11:29 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della 

+--------------------+-------------------+
|                NOME|                MSE|
+--------------------+-------------------+
|               Pavia| 0.2467808873129501|
|              Varese|0.37329949236192334|
|             Brescia| 0.1894754820259812|
|             Sondrio|0.20557668137501864|
|Monza e della Bri...| 0.4598376007640806|
|                Lodi|  0.253387690983838|
|                Como|0.47768401556946777|
|             Bergamo|0.23623985441206194|
|              Milano| 0.2996705055274734|
|             Cremona|0.21151254382924628|
|             Mantova| 0.2061497591309243|
|               Lecco| 0.4205445298584046|
+--------------------+-------------------+



Processing group: Lecco with 29862 data points===========>     (121 + 14) / 135]
Processing group: Pavia with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Brescia with 29862 data points
07:12:31 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
07:12:31 - cmdstanpy - INFO - Chain [1] start processing
07:12:31 - cmdstanpy - INFO - Chain [1] start processing
07:12:31 - cmdstanpy - INFO - Chain [1] start processing
07:12:31 - cmdstanpy - INFO - Chain [1] start processing
07:12:31 - cmdstanpy - INFO - Chain [1] start processing
07:12:41 - cmdstanpy - INFO - Chain [1] done processing
07:12:42 - cmdstanpy - INFO - Chain [1] done processing
07:12:44 - cmdstanpy - INFO - Chain [1] done processing
07:12:46 - cmdstanpy - INFO - Chain [1] done processing
07:12:46 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Milano with 298

+--------------------+-------------------+-------------------+
|                NOME|                MSE|               RMSE|
+--------------------+-------------------+-------------------+
|               Pavia| 0.2467808873129501| 0.4967704573673339|
|              Varese|0.37329949236192334| 0.6109823993880047|
|             Brescia| 0.1894754820259812|0.43528781515909815|
|             Sondrio|0.20557668137501864| 0.4534056477096626|
|Monza e della Bri...| 0.4598376007640806|  0.678113265438806|
|                Lodi|  0.253387690983838| 0.5033762916386091|
|                Como|0.47768401556946777| 0.6911468842217751|
|             Bergamo|0.23623985441206194|0.48604511561383057|
|              Milano| 0.2996705055274734| 0.5474216889450704|
|             Cremona|0.21151254382924628|0.45990492912040665|
|             Mantova| 0.2061497591309243|0.45403717813734623|
|               Lecco| 0.4205445298584046| 0.6484940476661328|
+--------------------+-------------------+-------------

In [16]:
from pyspark.sql.functions import col, pow, sqrt

# Define cutoff dates
cutoff_dates = {
    "5 Days": "2021-07-05 23:00:00",
    "30 Days": "2021-07-30 23:00:00",
    "90 Days": "2021-09-28 23:00:00",
    "180 Days": "2021-12-27 23:00:00",
    "365 Days": "2022-06-29 23:00:00"
}

# Define a list to store the evaluation results
results = []

# Compute evaluation metrics for each time period and NOME
for period, cutoff in cutoff_dates.items():
    for nome in spark_forecast_NO2.select("NOME").distinct().rdd.flatMap(lambda x: x).collect():
        # Filter the forecast data based on the cutoff date and NOME
        forecast_data_NO2 = spark_forecast_NO2.filter((col("ds") <= cutoff) & (col("NOME") == nome))

        # Calculate MAE
        mae = forecast_data_NO2.select(abs(col("y") - col("yhat")).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MAE") \
                             .collect()[0]["MAE"]

        # Calculate MSE
        mse = forecast_data_NO2.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MSE") \
                             .collect()[0]["MSE"]

        # Calculate RMSE
        rmse = forecast_data_NO2.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                              .agg({"error": "mean"}) \
                              .withColumnRenamed("avg(error)", "MSE") \
                              .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE")) \
                              .collect()[0]["RMSE"]

        # Calculate coverage
        coverage = forecast_data_NO2.filter((col("y") >= col("yhat_lower")) & (col("y") <= col("yhat_upper"))) \
                                  .count() / forecast_data_NO2.count() * 100

        # Add the results to the list
        results.append((period, nome, mse, rmse, mae, coverage))

# Create a Spark DataFrame from the results list
result_df_NO2 = spark.createDataFrame(results, ["Time", "NOME", "MSE", "RMSE", "MAE", "Coverage"])

# Show the results
result_df_NO2.show(100)

07:58:05 - cmdstanpy - INFO - Chain [1] start processing         (0 + 96) / 135]
07:58:22 - cmdstanpy - INFO - Chain [1] done processing         (96 + 39) / 135]
Processing group: Pavia with 29862 data points                      (0 + 6) / 6]
Processing group: Brescia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
07:58:31 - cmdstanpy - INFO - Chain [1] start processing
07:58:31 - cmdstanpy - INFO - Chain [1] start processing
07:58:31 - cmdstanpy - INFO - Chain [1] start processing
07:58:31 - cmdstanpy - INFO - Chain [1] start processing
07:58:31 - cmdstanpy - INFO - Chain [1] start processing
07:58:31 - cmdstanpy - INFO - Chain [1] start processing
07:58:42 - cmdstanpy - INFO - Chain [1] done processing
07:58:43 - cmdstanpy - INFO - Chain [1] done processing
07:58:44 - cmdstanpy - INFO - Chain [1] done processing
07:58:4

08:08:07 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
08:08:09 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
08:08:16 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Lodi with 29862 data points==============>   (125 + 10) / 135]
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 29862 data points
08:08:50 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:08:50 - cmdstanpy - INFO - Chain [1] start processing
08:08:50 - cmdstanpy - INFO - Chain [1] start processing
08:08:50 - cmdstanpy - INFO - Chain [1] start processing
08:08:50 - cmdstanpy - INFO - Chain [1] start processing
08:08:50 - cmdstanpy - INFO - Chain [1] start processing
08:09:01 - cmdstanpy - INFO - Chain [1] done processing
08:09:01 - cm

08:13:12 - cmdstanpy - INFO - Chain [1] done processing
08:13:12 - cmdstanpy - INFO - Chain [1] done processing
08:13:18 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
08:13:23 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
Processing group: Cremona with 29862 data points==>            (102 + 33) / 135]
Processing group: Bergamo with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Pavia with 29862 data points
08:13:54 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:13:54 - cmdstanpy - INFO - Chain [1] start processing
08:13:54 - cmdstanpy - INFO - Chain [1] start processing
08:13:54 - cmdstanpy - INFO - Chain [1] start processing
08:13:54 - cmdstanpy - INFO - Chain [1] start processing
08:13:54 - cmdstanpy - INFO - Chain [1] start processing
08:14:04 - cmdstanpy - INFO - Chain [1] d

08:18:16 - cmdstanpy - INFO - Chain [1] done processing
08:18:21 - cmdstanpy - INFO - Chain [1] done processing
08:18:21 - cmdstanpy - INFO - Chain [1] done processing
08:18:24 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
08:18:32 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Lecco with 29862 data points=====>           (105 + 30) / 135]
Processing group: Brescia with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Pavia with 29862 data points
08:19:04 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:19:04 - cmdstanpy - INFO - Chain [1] start processing
08:19:04 - cmdstanpy - INFO - Chain [1] start processing
08:19:04 - cmdstanpy - INFO - Chain [1] start processing
08:19:04 - cmdstanpy - INFO - Chain [1] start processing
08:19:04 - cmdstanpy - INFO - Chain [1] 

08:23:29 - cmdstanpy - INFO - Chain [1] done processing
08:23:30 - cmdstanpy - INFO - Chain [1] done processing
08:23:36 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
08:23:38 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
08:23:44 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
Processing group: Lecco with 29862 data points======>          (107 + 28) / 135]
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Lodi with 29862 data points
08:24:15 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:24:15 - cmdstanpy - INFO - Chain [1] start processing
08:24:15 - cmdstanpy - INFO - Chain [1] start processing
08:24:15 - cmdstanpy - INFO - Chain [1] start processing
08:24:15 - cmdstanpy - INFO - Chain [1] start processing
08:24:15 - cmds

...

14:09:37 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
Processing group: Milano with 29862 data points
14:09:39 - cmdstanpy - INFO - Chain [1] start processing
14:09:40 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
Processing group: Sondrio with 29862 data points
Processing group: Como with 29862 data points
14:09:42 - cmdstanpy - INFO - Chain [1] start processing
14:09:43 - cmdstanpy - INFO - Chain [1] start processing
14:09:43 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Varese with 29862 data points
14:09:45 - cmdstanpy - INFO - Chain [1] start processing
14:09:49 - cmdstanpy - INFO - Chain [1] done processing
14:09:50 - cmdstanpy - INFO - Chain [1] done processing
14:09:54 - cmdstanpy - INFO - Chain [1] done processing
14:09:58 - cmdstanpy - INFO - Chain [1] done processing             (2 + 4) / 6]
14:10:00 - cmdstanpy - INFO - Chain [1] done proce

14:14:39 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
Processing group: Milano with 29862 data points
Processing group: Mantova with 29862 data points
14:14:43 - cmdstanpy - INFO - Chain [1] start processing
14:14:43 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Sondrio with 29862 data points
14:14:44 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
14:14:45 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Varese with 29862 data points
14:14:46 - cmdstanpy - INFO - Chain [1] start processing
14:14:48 - cmdstanpy - INFO - Chain [1] start processing
14:14:52 - cmdstanpy - INFO - Chain [1] done processing
14:14:53 - cmdstanpy - INFO - Chain [1] done processing
14:14:57 - cmdstanpy - INFO - Chain [1] done processing
14:14:59 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
14:15:00 - cmdstanpy - INFO - Chain [1] done proce

Processing group: Como with 29862 data points
Processing group: Milano with 29862 data points
14:19:41 - cmdstanpy - INFO - Chain [1] done processing
14:19:41 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
14:19:42 - cmdstanpy - INFO - Chain [1] done processing
14:19:42 - cmdstanpy - INFO - Chain [1] start processing
14:19:42 - cmdstanpy - INFO - Chain [1] start processing
14:19:43 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Varese with 29862 data points
Processing group: Sondrio with 29862 data points
14:19:50 - cmdstanpy - INFO - Chain [1] start processing
14:19:50 - cmdstanpy - INFO - Chain [1] start processing
14:19:50 - cmdstanpy - INFO - Chain [1] done processing
14:19:51 - cmdstanpy - INFO - Chain [1] done processing
14:19:56 - cmdstanpy - INFO - Chain [1] done processing
14:19:58 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5) / 6]
14:20:01 - cmdstanpy - INFO - Chain [1] done processing  

14:24:45 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
14:24:46 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Milano with 29862 data points
Processing group: Mantova with 29862 data points
14:24:48 - cmdstanpy - INFO - Chain [1] start processing
14:24:48 - cmdstanpy - INFO - Chain [1] start processing
14:24:48 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
Processing group: Varese with 29862 data points
14:24:52 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Sondrio with 29862 data points
14:24:53 - cmdstanpy - INFO - Chain [1] start processing
14:24:55 - cmdstanpy - INFO - Chain [1] start processing
14:24:58 - cmdstanpy - INFO - Chain [1] done processing
14:24:59 - cmdstanpy - INFO - Chain [1] done processing
14:25:00 - cmdstanpy - INFO - Chain [1] done processing
14:25:05 - cmdstanpy - INFO - Chain [1] done processing             (1 + 5)

+--------+--------------------+-------------------+-------------------+-------------------+-----------------+
|    Time|                NOME|                MSE|               RMSE|                MAE|         Coverage|
+--------+--------------------+-------------------+-------------------+-------------------+-----------------+
|  5 Days|               Pavia|0.19131513199240407| 0.4373958527380022| 0.3483612365238471|95.27985074626866|
|  5 Days|              Varese| 0.3009560080438256|   0.54859457529566| 0.4339283868465191|94.51026119402985|
|  5 Days|             Brescia|0.15070890235130513| 0.3882124448691787| 0.3081973611808969| 95.2518656716418|
|  5 Days|             Sondrio| 0.1367751503041035|0.36983124571093706| 0.2842538687172462| 93.8945895522388|
|  5 Days|Monza e della Bri...|  0.318099240477163| 0.5640028727561262|0.43519240177080676| 93.9132462686567|
|  5 Days|                Lodi| 0.2021979230606364|0.44966423369069103|0.35531573046565385|95.04664179104478|
|  5 Days|

In [ ]:
result_df_NO2_pd = result_df_NO2.pandas_api()
result_df_NO2_pd

O3 

In [18]:
from pyspark.sql.functions import *
from time import time
import matplotlib.pyplot as plt
import os

data_O3_prophet = data_O3.groupBy("NOME", "original_date_time").agg(avg("c_O3").alias("avg_c_O3"))
data_O3_prophet.show()

# Apply logarithmic transformation to the aggregated column avg_c_O3
data_O3_prophet = data_O3_prophet.withColumn("avg_c_O3_log", log1p("avg_c_O3"))

data_O3_prophet_u = data_O3_prophet.withColumnRenamed("avg_c_O3_log", "y").withColumnRenamed("original_date_time", "ds")

data_O3_prophet = data_O3_prophet_u.withColumn("ds", to_timestamp("ds", "yyyy-MM-dd HH:mm:ss")) \
                                       .withColumn("ds", from_utc_timestamp("ds", "Europe/Rome"))

data_O3_prophet.printSchema()

def train_and_forecast(group):
    
    print(f"Processing group: {group['NOME'].iloc[0]} with {len(group)} data points")
    
    group["ds"] = pd.to_datetime(group["ds"])

    m = Prophet(seasonality_mode="additive",
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=True,
                    interval_width=0.95)
    
    m.fit(group[group['ds'] <= cutoff_date])
    
    forecast = m.predict(group)
    
    path = '/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/O3/'

    if not os.path.exists(path):
        os.makedirs(path)
    
   # try:
   #     fig1 = m.plot(forecast)
   #     plt.ylabel("Concentration (µg/m3)", fontweight="bold")
   #     plt.xlabel("Date", fontweight="bold")
   #     fig1.suptitle(f"O3 Concentration Forecast for {group['NOME'].iloc[0]}")
    #    fig1.legend(['Actual', 'Forecasted'], loc='upper right')
    #    fig1.subplots_adjust(top=0.9)
        
    #    fig2 = m.plot_components(forecast)
    #    fig2.suptitle(f"O3 Concentration Component Forecast for {group['NOME'].iloc[0]}")
     #   fig2.subplots_adjust(top=0.9)

    #    fig1.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/O3/{group['NOME'].iloc[0]}_prophet_forecast.png")
    #    fig2.savefig(f"/afs/enea.it/por/user/nafis/PFS/por/Thesis/codes/prophet/new/O3/{group['NOME'].iloc[0]}_prophet_components.png")
   # except Exception as e:
    #    print(f"Error generating plots for group {group['NOME'].iloc[0]}: {str(e)}")
    
    
    f_pd = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','trend', 'yearly', 'weekly', 'daily']].set_index("ds")
 
    group_pd = group[['ds', 'NOME', 'y']].set_index("ds")
    
    result_pd = f_pd.join( group_pd, how = "left")
    result_pd.reset_index(level=0, inplace=True)
    
    result_pd['NOME'] = group['NOME'].iloc[0]
    
    return result_pd[['ds', 'NOME', 'y', 'yhat', 'yhat_upper', 'yhat_lower','trend', 'yearly', 'weekly', 'daily']] 

result_schema =StructType([
    StructField('ds',TimestampType()),
    StructField('NOME', StringType()),
    StructField('y', DoubleType()),
    StructField('yhat',FloatType()),
    StructField('yhat_upper',FloatType()),
    StructField('yhat_lower',FloatType()),
    StructField('trend',FloatType()),
    StructField('yearly',FloatType()),
    StructField('weekly',FloatType()),
    StructField('daily',FloatType())
  ])


# create train and test data based on cutoff date
cutoff_date = "2021-06-30 23:00:00"

data_O3_prophet_train = data_O3_prophet.filter(col('ds') <= cutoff_date)
data_O3_prophet_test = data_O3_prophet.filter((col('ds') > cutoff_date))

max_date = data_O3_prophet_test.agg(max('ds')).collect()[0][0]

period = data_O3_prophet_test.count()

# Start time
start_time = time()
# Train and forecast by ticker 
spark_forecast_O3 = data_O3_prophet.groupBy("NOME").applyInPandas(train_and_forecast, schema=result_schema)
# Processing time
print('The time used for the Spark forecast is ', time()-start_time)

spark_forecast_O3.show()


+-------+-------------------+------------------+
|   NOME| original_date_time|          avg_c_O3|
+-------+-------------------+------------------+
|Sondrio|2019-11-24 14:00:00| 54.41587480904522|
| Varese|2020-08-04 06:00:00|56.921168171052614|
| Varese|2021-07-11 17:00:00|100.52956123684211|
|Brescia|2019-11-23 07:00:00|        54.5038949|
| Varese|2021-06-27 10:00:00|114.93741321052627|
|  Pavia|2021-06-26 01:00:00| 96.90419869518705|
| Varese|2021-06-26 05:00:00| 76.72459271052632|
| Milano|2021-11-28 12:00:00|43.479280416666654|
| Varese|2022-05-29 00:00:00| 78.74115980921054|
|   Como|2021-05-17 10:00:00|112.92797683950621|
|Mantova|2019-11-23 22:00:00| 51.07148930344828|
|  Lecco|2022-06-11 19:00:00| 128.0819811372549|
|  Pavia|2022-06-11 03:00:00|  87.2764229197861|
| Varese|2021-06-26 15:00:00|138.17697368421054|
|Sondrio|2019-11-24 04:00:00| 52.91707371859296|
|   Lodi|2020-08-04 21:00:00|32.878370431372545|
|Brescia|2021-05-17 12:00:00| 107.4413135833333|
|  Lecco|2022-06-11 

The time used for the Spark forecast is  0.03859090805053711


Processing group: Pavia with 29862 data points                      (0 + 1) / 1]
09:16:46 - cmdstanpy - INFO - Chain [1] start processing
09:17:01 - cmdstanpy - INFO - Chain [1] done processing


+-------------------+-----+------------------+---------+----------+----------+---------+----------+-------------+------------+
|                 ds| NOME|                 y|     yhat|yhat_upper|yhat_lower|    trend|    yearly|       weekly|       daily|
+-------------------+-----+------------------+---------+----------+----------+---------+----------+-------------+------------+
|2019-01-01 01:00:00|Pavia| 3.305281714293108|3.2413406| 3.9912138| 2.5123222|   4.6563|-1.1168591| -0.044930752| -0.25316966|
|2019-01-01 02:00:00|Pavia| 3.255936267711403|3.2400022|  3.978302| 2.4472563|4.6550574|-1.1164461|  -0.04469888| -0.25391048|
|2019-01-01 03:00:00|Pavia|3.2157234909407153|3.2511473| 4.0254087| 2.4422712| 4.653815|-1.1160367| -0.043995254|  -0.2426355|
|2019-01-01 04:00:00|Pavia| 3.280340899366269|3.2695818|  4.036458|  2.496477| 4.652572|-1.1156307| -0.042856727| -0.22450265|
|2019-01-01 05:00:00|Pavia| 3.397328081682102| 3.286361|  4.020008| 2.5541494|4.6513295|-1.1152284|  -0.0413240

Processing group: Varese with 29862 data points


In [49]:
from pyspark.sql.functions import col
from pyspark.sql.functions import pow, sqrt
from prophet.diagnostics import performance_metrics


# Calculate MAE
mae_O3 = spark_forecast_O3.select(col("NOME"), abs(col("y") - col("yhat")).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MAE")


# Calculate MSE
mse_O3 = spark_forecast_O3.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                    .groupBy("NOME") \
                    .agg({"error": "mean"}) \
                    .withColumnRenamed("avg(error)", "MSE")

# Calculate RMSE
rmse_O3 = spark_forecast_O3.select(col("NOME"), pow(col("y") - col("yhat"), 2).alias("error")) \
                     .groupBy("NOME") \
                     .agg({"error": "mean"}) \
                     .withColumnRenamed("avg(error)", "MSE") \
                     .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE"))

# Show the results
mae_O3.show()
mse_O3.show()
rmse_O3.show()

08:48:48 - cmdstanpy - INFO - Chain [1] start processing         (0 + 96) / 135]
08:49:04 - cmdstanpy - INFO - Chain [1] done processing          (1 + 96) / 135]
Processing group: Lecco with 29862 data points                      (0 + 6) / 6]
Processing group: Brescia with 29862 data pointsProcessing group: Lodi with 29862 data points

Processing group: Bergamo with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
08:49:31 - cmdstanpy - INFO - Chain [1] start processing
08:49:31 - cmdstanpy - INFO - Chain [1] start processing
08:49:31 - cmdstanpy - INFO - Chain [1] start processing
08:49:31 - cmdstanpy - INFO - Chain [1] start processing
08:49:31 - cmdstanpy - INFO - Chain [1] start processing
08:49:31 - cmdstanpy - INFO - Chain [1] start processing
08:49:50 - cmdstanpy - INFO - Chain [1] done processing
08:49:54 - cmdstanpy - INFO - Chain [1] done processing
08:49:55 - cmdstanpy - INFO - Chain [1] done processing
08:49:5

+--------------------+-------------------+
|                NOME|                MAE|
+--------------------+-------------------+
|               Pavia| 0.3229165492435026|
|              Varese| 0.4728466773174461|
|             Brescia| 0.2173243990910875|
|             Sondrio| 0.1278501214963288|
|Monza e della Bri...| 0.7022569112041227|
|                Lodi| 0.5231215317659541|
|                Como| 0.2831250259508965|
|             Bergamo|  0.222936776439256|
|              Milano| 0.6452928977618836|
|             Cremona|0.42934692514482625|
|             Mantova|0.40420891885509225|
|               Lecco| 0.2739203188267231|
+--------------------+-------------------+



Processing group: Pavia with 29862 data points=================>(133 + 2) / 135]
Processing group: Cremona with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Bergamo with 29862 data points
08:51:16 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:51:16 - cmdstanpy - INFO - Chain [1] start processing
08:51:16 - cmdstanpy - INFO - Chain [1] start processing
08:51:17 - cmdstanpy - INFO - Chain [1] start processing
08:51:17 - cmdstanpy - INFO - Chain [1] start processing
08:51:17 - cmdstanpy - INFO - Chain [1] start processing
08:51:34 - cmdstanpy - INFO - Chain [1] done processing
08:51:34 - cmdstanpy - INFO - Chain [1] done processing
08:51:40 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Milano with 29862 data points
Processing group: Monza e della Brianza with 29862 data points
08:51:43 - cmdstanpy - INFO - Chai

+--------------------+--------------------+
|                NOME|                 MSE|
+--------------------+--------------------+
|               Pavia|  0.1993724652417141|
|              Varese|  0.4310561580851715|
|             Brescia| 0.08632067723995296|
|             Sondrio|0.028273050823911544|
|Monza e della Bri...|  0.8366827085322027|
|                Lodi|  0.5093961543162513|
|                Como| 0.15663757090022082|
|             Bergamo| 0.09312414477486554|
|              Milano|  0.7332012798796659|
|             Cremona| 0.36016568090933426|
|             Mantova| 0.32017013310293657|
|               Lecco| 0.15630784041848453|
+--------------------+--------------------+



Processing group: Pavia with 29862 data points===>             (100 + 35) / 135]
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Lodi with 29862 data points
08:53:11 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
08:53:11 - cmdstanpy - INFO - Chain [1] start processing
08:53:11 - cmdstanpy - INFO - Chain [1] start processing
08:53:11 - cmdstanpy - INFO - Chain [1] start processing
08:53:11 - cmdstanpy - INFO - Chain [1] start processing
08:53:11 - cmdstanpy - INFO - Chain [1] start processing
08:53:28 - cmdstanpy - INFO - Chain [1] done processing
08:53:32 - cmdstanpy - INFO - Chain [1] done processing
08:53:33 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Milano with 29862 data points
08:53:37 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Monza e della Brianza 

+--------------------+--------------------+-------------------+
|                NOME|                 MSE|               RMSE|
+--------------------+--------------------+-------------------+
|               Pavia|  0.1993724652417141|0.44651143909390956|
|              Varese|  0.4310561580851715| 0.6565486715279922|
|             Brescia| 0.08632067723995296| 0.2938038073952633|
|             Sondrio|0.028273050823911544| 0.1681459212229412|
|Monza e della Bri...|  0.8366827085322027|  0.914703617863296|
|                Lodi|  0.5093961543162513| 0.7137199410947205|
|                Como| 0.15663757090022082| 0.3957746466112007|
|             Bergamo| 0.09312414477486554|0.30516248913466665|
|              Milano|  0.7332012798796659| 0.8562717325006507|
|             Cremona| 0.36016568090933426| 0.6001380515425883|
|             Mantova| 0.32017013310293657| 0.5658357828053441|
|               Lecco| 0.15630784041848453| 0.3953578637367474|
+--------------------+------------------

In [19]:
from pyspark.sql.functions import col, pow, sqrt

# Define cutoff dates
cutoff_dates = {
    "5 Days": "2021-07-05 23:00:00",
    "30 Days": "2021-07-30 23:00:00",
    "90 Days": "2021-09-28 23:00:00",
    "180 Days": "2021-12-27 23:00:00",
    "365 Days": "2022-06-29 23:00:00"
}

# Define a list to store the evaluation results
results = []

# Compute evaluation metrics for each time period and NOME
for period, cutoff in cutoff_dates.items():
    for nome in spark_forecast_O3.select("NOME").distinct().rdd.flatMap(lambda x: x).collect():
        # Filter the forecast data based on the cutoff date and NOME
        forecast_data_O3 = spark_forecast_O3.filter((col("ds") <= cutoff) & (col("NOME") == nome))

        # Calculate MAE
        mae = forecast_data_O3.select(abs(col("y") - col("yhat")).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MAE") \
                             .collect()[0]["MAE"]

        # Calculate MSE
        mse = forecast_data_O3.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                             .agg({"error": "mean"}) \
                             .withColumnRenamed("avg(error)", "MSE") \
                             .collect()[0]["MSE"]

        # Calculate RMSE
        rmse = forecast_data_O3.select(pow(col("y") - col("yhat"), 2).alias("error")) \
                              .agg({"error": "mean"}) \
                              .withColumnRenamed("avg(error)", "MSE") \
                              .withColumn("RMSE", sqrt(col("MSE")).alias("RMSE")) \
                              .collect()[0]["RMSE"]

        # Calculate coverage
        coverage = forecast_data_O3.filter((col("y") >= col("yhat_lower")) & (col("y") <= col("yhat_upper"))) \
                                  .count() / forecast_data_O3.count() * 100

        # Add the results to the list
        results.append((period, nome, mse, rmse, mae, coverage))

# Create a Spark DataFrame from the results list
result_df_O3 = spark.createDataFrame(results, ["Time", "NOME", "MSE", "RMSE", "MAE", "Coverage"])

# Show the results
result_df_O3.show(100)


09:17:10 - cmdstanpy - INFO - Chain [1] start processing         (0 + 96) / 135]
Processing group: Lodi with 29862 data points                       (0 + 6) / 6]
Processing group: Bergamo with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Lecco with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Brescia with 29862 data points
09:17:34 - cmdstanpy - INFO - Chain [1] start processing
09:17:34 - cmdstanpy - INFO - Chain [1] start processing
09:17:34 - cmdstanpy - INFO - Chain [1] start processing
09:17:34 - cmdstanpy - INFO - Chain [1] start processing
09:17:34 - cmdstanpy - INFO - Chain [1] start processing
09:17:34 - cmdstanpy - INFO - Chain [1] start processing
09:17:39 - cmdstanpy - INFO - Chain [1] done processing
09:17:51 - cmdstanpy - INFO - Chain [1] done processing
09:17:54 - cmdstanpy - INFO - Chain [1] done processing
09:17:56 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Milano with 2

09:23:52 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
09:23:59 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
09:24:08 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Bergamo with 29862 data points==>            (102 + 33) / 135]
Processing group: Lecco with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Brescia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Lodi with 29862 data points
09:24:37 - cmdstanpy - INFO - Chain [1] start processing            (0 + 6) / 6]
09:24:37 - cmdstanpy - INFO - Chain [1] start processing
09:24:37 - cmdstanpy - INFO - Chain [1] start processing09:24:37 - cmdstanpy - INFO - Chain [1] start processing

09:24:37 - cmdstanpy - INFO - Chain [1] start processing
09:24:37 - cmdstanpy - INFO - Chain [1] start processing
09:24:55 - cmdstanpy - INFO - Chain [1] done processing
09:24:58 - cmdst

09:37:49 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
09:37:51 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
09:37:56 - cmdstanpy - INFO - Chain [1] start processing            (1 + 5) / 6]
09:37:59 - cmdstanpy - INFO - Chain [1] done processing
09:38:01 - cmdstanpy - INFO - Chain [1] done processing
09:38:08 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
09:38:16 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
09:38:23 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Lecco with 29862 data points                     (7 + 7) / 14]
Processing group: Brescia with 29862 data points
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Lodi with 29862 data points
Processing group: Bergamo with 29862 data points
09:38:54 - cmdstanpy - INFO - Chai

09:44:37 - cmdstanpy - INFO - Chain [1] start processing
09:44:41 - cmdstanpy - INFO - Chain [1] done processing
09:44:43 - cmdstanpy - INFO - Chain [1] done processing
09:44:44 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Mantova with 29862 data points
09:44:49 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
09:44:52 - cmdstanpy - INFO - Chain [1] start processing            (1 + 5) / 6]
09:44:56 - cmdstanpy - INFO - Chain [1] done processing
09:44:57 - cmdstanpy - INFO - Chain [1] done processing
09:45:07 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
09:45:14 - cmdstanpy - INFO - Chain [1] done processing
09:45:19 - cmdstanpy - INFO - Chain [1] done processing             (4 + 2) / 6]
Processing group: Lecco with 29862 data points===========>     (120 + 15) / 135]
Processing group: Pavia with 29862 data points
Processing group: Cremona with 29862 data points
Processing group: Bergamo with 2986

09:51:48 - cmdstanpy - INFO - Chain [1] done processing
09:51:48 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Sondrio with 29862 data points
09:51:52 - cmdstanpy - INFO - Chain [1] done processing
09:51:53 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points                    (1 + 5) / 6]
09:51:56 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
09:52:00 - cmdstanpy - INFO - Chain [1] start processing
09:52:02 - cmdstanpy - INFO - Chain [1] done processing
09:52:03 - cmdstanpy - INFO - Chain [1] done processing
09:52:14 - cmdstanpy - INFO - Chain [1] done processing             (3 + 3) / 6]
09:52:20 - cmdstanpy - INFO - Chain [1] done processing
09:52:28 - cmdstanpy - INFO - Chain [1] done processing===>         (5 + 1) / 6]
Processing group: Brescia with 29862 data points==>            (101 + 34) / 135]
Processing group: Lecco with 29862 data points
Processing group: Cremona w

...

17:27:30 - cmdstanpy - INFO - Chain [1] done processing
17:27:30 - cmdstanpy - INFO - Chain [1] done processing
17:27:35 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Monza e della Brianza with 29862 data points
Processing group: Milano with 29862 data points
17:27:38 - cmdstanpy - INFO - Chain [1] start processing
17:27:39 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Varese with 29862 data points
17:27:43 - cmdstanpy - INFO - Chain [1] start processing
17:27:43 - cmdstanpy - INFO - Chain [1] done processing
17:27:49 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Sondrio with 29862 data points
17:27:50 - cmdstanpy - INFO - Chain [1] done processing
17:27:50 - cmdstanpy - INFO - Chain [1] done processing
17:27:51 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points                    (1 + 5) / 6]
Processing group: Como with 29862 data points
17:27:58 - cmdstanpy - INFO - Chain [1] start

17:34:38 - cmdstanpy - INFO - Chain [1] done processing
17:34:43 - cmdstanpy - INFO - Chain [1] done processing
17:34:43 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Milano with 29862 data points
17:34:46 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Monza e della Brianza with 29862 data points
Processing group: Varese with 29862 data points
17:34:50 - cmdstanpy - INFO - Chain [1] done processing
17:34:51 - cmdstanpy - INFO - Chain [1] start processing
17:34:51 - cmdstanpy - INFO - Chain [1] start processing
17:34:55 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Sondrio with 29862 data points
17:34:57 - cmdstanpy - INFO - Chain [1] done processing
17:34:58 - cmdstanpy - INFO - Chain [1] done processing
17:34:59 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
17:35:03 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
17:35:05 - cmdstanpy

17:41:41 - cmdstanpy - INFO - Chain [1] done processing
17:41:44 - cmdstanpy - INFO - Chain [1] done processing
17:41:47 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Varese with 29862 data points
17:41:49 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Milano with 29862 data points
17:41:52 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Monza e della Brianza with 29862 data points
17:41:55 - cmdstanpy - INFO - Chain [1] start processing
17:41:55 - cmdstanpy - INFO - Chain [1] done processing
17:42:00 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Sondrio with 29862 data points
17:42:02 - cmdstanpy - INFO - Chain [1] done processing
17:42:03 - cmdstanpy - INFO - Chain [1] start processing
17:42:04 - cmdstanpy - INFO - Chain [1] done processing
17:42:04 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Mantova with 29862 data points
17:42:09 - cmdstanpy - INFO - Chain [1] start processing
Processing

17:48:47 - cmdstanpy - INFO - Chain [1] done processing
17:48:51 - cmdstanpy - INFO - Chain [1] done processing
17:48:52 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Milano with 29862 data points
17:48:56 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Monza e della Brianza with 29862 data points
Processing group: Varese with 29862 data points
17:48:59 - cmdstanpy - INFO - Chain [1] start processing
17:49:00 - cmdstanpy - INFO - Chain [1] done processing
17:49:00 - cmdstanpy - INFO - Chain [1] start processing
17:49:01 - cmdstanpy - INFO - Chain [1] done processing
Processing group: Sondrio with 29862 data points
17:49:07 - cmdstanpy - INFO - Chain [1] done processing
17:49:07 - cmdstanpy - INFO - Chain [1] done processing
17:49:08 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Mantova with 29862 data points
17:49:10 - cmdstanpy - INFO - Chain [1] start processing
Processing group: Como with 29862 data points
17:49:16 - cmdstanpy

+--------+--------------------+--------------------+-------------------+-------------------+-----------------+
|    Time|                NOME|                 MSE|               RMSE|                MAE|         Coverage|
+--------+--------------------+--------------------+-------------------+-------------------+-----------------+
|  5 Days|               Pavia| 0.14559854778981265| 0.3815737776496344| 0.2693439457781378|94.52891791044776|
|  5 Days|              Varese|  0.3554808029423752| 0.5962221087332935| 0.4198147461506301|93.26958955223881|
|  5 Days|             Brescia| 0.06339639288077979| 0.2517864032881438|0.18642260282297649|94.35167910447761|
|  5 Days|             Sondrio|0.023171084152118816|0.15222051160115976|0.11375783668833143| 95.3731343283582|
|  5 Days|Monza e della Bri...|  0.6896858164611583| 0.8304732484921825| 0.6370377005555615|93.71735074626866|
|  5 Days|                Lodi|  0.3897578823513894| 0.6243059204840119| 0.4495188944619429|92.74720149253731|
|

In [ ]:
result_df_O3_pd = result_df_O3.pandas_api()
result_df_O3_pd

In [16]:
spark.stop()